In [ ]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import numpy as np
import matplotlib.pylab as plt
import matplotlib
%matplotlib qt
from numpy import linalg as la
from scipy.optimize import fsolve
from scipy import linalg as scpla
# import seaborn as sb
from mpl_toolkits.axes_grid1 import make_axes_locatable
from cmath import *
from mpl_toolkits.mplot3d import Axes3D
from scipy.optimize import fsolve,leastsq 
from math import tanh,cosh
import math
import time

from matplotlib.patches import Ellipse
import matplotlib.transforms as transforms

# from sympy import *
from scipy.linalg import schur, eigvals
from scipy.special import comb, perm

extras_require = {'PLOT':['matplotlib>=1.1.1,<3.0']},

## help functions

In [ ]:
def generate_localstatsmat_eig_desym(Nparams,Grand,rho):
    '''
    Input
      Grand 4, gee,gei,gie,gii
      Nparams: number of E and I neurons (same)
    '''
    NE,NI=Nparams[0],Nparams[1]
    N=NE+NI
    xaxiss,yaxiss =np.arange(NE*2),np.arange(NE*2)
    Xax,Yax=np.meshgrid(xaxiss,yaxiss)
    idxwhere = np.where(Xax>Yax)
    xee,xei,xie,xii=Grand[0],Grand[1],Grand[2],Grand[3]
    X,Xsym,Xasym=np.zeros((N,N)),np.zeros((N,N)),np.zeros((N,N))
    X[:NE,:NE],X[NE:,NE:]=np.random.randn(NE,NE),np.random.randn(NE,NE)
    X[:NE,NE:],X[NE:,:NE]=np.random.randn(NE,NE),np.random.randn(NE,NE)
    XT = X.copy().T

    Xsym = np.sqrt(rho)*X.copy()
    Xsym[idxwhere]=np.sqrt(rho)*XT[idxwhere]
    
    Xasym=np.sqrt(1-rho)*np.random.randn(NE*2,NE*2)
    X=Xsym.copy()+Xasym.copy()
    # components
    Xsym[:NE,:NE],Xsym[NE:,NE:]=xee*Xsym[:NE,:NE]/np.sqrt(NE),xii*Xsym[NE:,NE:]/np.sqrt(NI)
    Xsym[:NE,NE:],Xsym[NE:,:NE]=xei*Xsym[:NE,NE:]/np.sqrt(NE),xie*Xsym[NE:,:NE]/np.sqrt(NI)
    Xasym[:NE,:NE],Xasym[NE:,NE:]=xee*Xasym[:NE,:NE]/np.sqrt(NE),xii*Xasym[NE:,NE:]/np.sqrt(NI)
    Xasym[:NE,NE:],Xasym[NE:,:NE]=xei*Xasym[:NE,NE:]/np.sqrt(NE),xie*Xasym[NE:,:NE]/np.sqrt(NI)
    X=Xsym.copy()+Xasym.copy()
    # theoretical M (grandom) matrix
    gmat = np.array([[xee**2,xei**2],[xie**2,xii**2]])
    # first do not multiply at
    gaverage=0
    for i in range(2):
        for j in range(2):
            gaverage+=gmat[i,j]/2 # ntype=2
    gaverage=np.sqrt(gaverage)
    # for i in range(2):
    #     gmat[:,i]*=at[i]
    eigvgm,eigvecgm=la.eig(gmat)
    
    # properties
    Gamp=np.zeros((2,2))
    Gamp[0,0],Gamp[0,1],Gamp[1,0],Gamp[1,1]=xee,xei,xie,xii
    return (X,Xsym,Xasym,Gamp,eigvgm,eigvecgm,gaverage)

def generate_localstatsmat_eig(Nparams,Grand):
    '''
    Input 
      Grand 4, gee,gei,gie,gii
      Nparams: number of E and I neurons (same)
    '''
    xee,xei,xie,xii=Grand[0],Grand[1],Grand[2],Grand[3]
    NE,NI=Nparams[0],Nparams[1]
    N=NE+NI
    at = Nparams/N   
    X=np.zeros((N,N))
    X[:NE,:NE],X[NE:,NE:]=np.random.randn(NE,NE),np.random.randn(NE,NE)
    X[:NE,NE:],X[NE:,:NE]=np.random.randn(NE,NE),np.random.randn(NE,NE)
    X[:NE,:NE],X[NE:,NE:]=xee*X[:NE,:NE]/np.sqrt(NE),xii*X[NE:,NE:]/np.sqrt(NI)
    X[:NE,NE:],X[NE:,:NE]=xei*X[:NE,NE:]/np.sqrt(NE),xie*X[NE:,:NE]/np.sqrt(NI)
    # theoretical M (grandom) matrix 
    gmat = np.array([[xee**2,xei**2],[xie**2,xii**2]])
    # first do not multiply at
    gaverage=0
    for i in range(2):
        for j in range(2):
            gaverage+=gmat[i,j]/2 # ntype=2
    gaverage=np.sqrt(gaverage)
    eigvgm,eigvecgm=la.eig(gmat)
    # properties
    Gamp=np.zeros((2,2))
    Gamp[0,0],Gamp[0,1],Gamp[1,0],Gamp[1,1]=xee,xei,xie,xii

    return (X,Gamp,eigvgm,eigvecgm,gaverage)

def generate_meanmat_eig(Nparams,JEE,JIE,JEI,JII):
    # mean value 
    # first use rank-1 structure
    NE,NI=Nparams[0],Nparams[1]
    N=NE+NI
    at = Nparams/N   
    Am=np.zeros((N,N))
    Am[:NE,:NE],Am[:NE,NE:]=JEE/NE,-JEI/NI
    Am[NE:,:NE],Am[NE:,NE:]=JIE/NE,-JII/NI

    Jsv=np.zeros((2,2))
    Jsv[0,0],Jsv[0,1],Jsv[1,0],Jsv[1,1]=JEE,JEI,JIE,JII

    return (Am,Jsv)#,ua,svam,va)

# generate gmat, same gavg
def gmatamplitude_eig(gavgfix,typenum):
    Amplit = gavgfix*typenum
    numsample = typenum**2
    Amplitg= np.zeros(numsample)
    idxc=0
    while (1):
      if idxc>=numsample:
        Amplitg[numsample-1]=1.0-np.sum(Amplitg[:numsample-1])
        break
      p=np.random.random(1)
      Amplitg[idxc]=np.minimum(p,1.0-np.sum(Amplitg))
      if np.sum(Amplitg)>1.0:
        continue 
      elif np.sum(Amplitg)==1.0:
        break
      else:
        idxc +=1
        # Amplitg[idxc]=np.min(p,1.0-np.sum(Amplitg))
    # Amplitg=0
    Amplitg*=Amplit
    Amplitg=np.sqrt(Amplitg)
    return Amplitg

def confidence_ellipse(x, y, ax, n_std=3.0, facecolor='none', **kwargs):
    """
    Create a plot of the covariance confidence ellipse of `x` and `y`

    Parameters
    ----------
    x, y : array_like, shape (n, )
        Input data.

    ax : matplotlib.axes.Axes
        The axes object to draw the ellipse into.

    n_std : float
        The number of standard deviations to determine the ellipse's radiuses.

    Returns
    -------
    matplotlib.patches.Ellipse

    Other parameters
    ----------------
    kwargs : `~matplotlib.patches.Patch` properties
    """
    if x.size != y.size:
        raise ValueError("x and y must be the same size")

    cov = np.cov(x, y)
    print(np.shape(cov))
    pearson = cov[0, 1]/np.sqrt(cov[0, 0] * cov[1, 1])
    # Using a special case to obtain the eigenvalues of this
    # two-dimensionl dataset.
    ell_radius_x = np.sqrt(1 + pearson)
    ell_radius_y = np.sqrt(1 - pearson)
    ellipse = Ellipse((0, 0),
        width=ell_radius_x * 2,
        height=ell_radius_y * 2,
        facecolor=facecolor,
        **kwargs)

    # Calculating the stdandard deviation of x from
    # the squareroot of the variance and multiplying
    # with the given number of standard deviations.
    scale_x = np.sqrt(cov[0, 0]) * n_std
    mean_x = np.mean(x)

    # calculating the stdandard deviation of y ...
    scale_y = np.sqrt(cov[1, 1]) * n_std
    mean_y = np.mean(y)

    transf = transforms.Affine2D() \
        .rotate_deg(45) \
        .scale(scale_x, scale_y) \
        .translate(mean_x, mean_y)

    ellipse.set_transform(transf + ax.transData)
    return ax.add_patch(ellipse)

import math
import cmath
def ThreeSquare(x):
    if x.imag == 0:
        #实数
        m = x.real
        ans = -math.pow(-m,1/3) if m<0 else math.pow(m,1/3)
    else:
        #复数
        ans = x**(1/3)
    return ans

def RoundAns(x,num):
    if x.imag == 0:
        m = x.real
        ans = round(m,num)
    else:
        m = round(x.real,num)
        n = round(x.imag,num)
        ans = complex(m,n)
    return ans
    
def Cubic(args):
    a,b,c,d = args
    p = c/a-b**2/(3*a**2)
    q = d/a+2*b**3/(27*a**3)-b*c/(3*a**2)
    w = complex(-0.5,(3**0.5)/2)
    ww = complex(-0.5,-(3**0.5)/2)
    A = cmath.sqrt((q/2)**2+(p/3)**3)
    B = ThreeSquare(-q/2+A)
    C = ThreeSquare(-q/2-A)
    y1 = B+C
    y2 = w*B+ww*C
    y3 = ww*B+w*C
    D = b/(3*a)
    roots=[RoundAns(y1-D,6),RoundAns(y2-D,6),RoundAns(y3-D,6)]
    return roots


def gradtanh(xorg):
    gradx = xorg.copy()
    nneuron,nknum=np.shape(gradx)[0],np.shape(gradx)[1]
    for i in range(nneuron):
        for j in range(nknum):
            gradx[i,j]=4/(np.exp(-xorg[i,j])+np.exp(xorg[i,j]))**2
    return gradx

## generate iid Gaussian perturbed matrix

In [ ]:
'''
Cases 1,2 without symmetric connectivity
Focus on standard deviation (variance)
Focus on second moment (self)
'''
# generate mean matrix
nrank=1
Nt=np.array([750,750])
NE,NI=Nt[0],Nt[1]
N=NE+NI
Nparams=np.array([NE,NI])
nrank,ntrial,neta,ngavg=1,20,10,10
## for all casesq
xticks = np.linspace(-4.0,3.0,5)
xlims = [-4.1,3.1]
yticks = np.linspace(-1.3,1.3,3)
ylims = [-1.31,1.31]
theta = np.linspace(0, 2 * np.pi, 200)

''' C0 rules '''
Xtest = np.zeros((NE*2,NE*2))
Xtest[:NE,:NE]=1
idxEE,idyEE=np.where(Xtest>0)
lowinEE=np.where(idxEE<idyEE)
upinEE=np.where(idxEE>idyEE)
idxUpEE,idyUpEE,idxDownEE,idyDownEE=idxEE[lowinEE],idyEE[lowinEE],idxEE[upinEE],idyEE[upinEE]
Xrep = np.zeros_like(Xtest)
Xrep[idxUpEE,idyUpEE]=1
idxUpII,idyUpII,idxDownII,idyDownII=idxUpEE+NE,idyUpEE+NE,idxDownEE+NE,idyDownEE+NE

''' ---------- method 2 ------------------ '''
''' single eta '''
symetaseries = np.linspace(0.1,1.0,10) # a series of symmetricity value
etaset=0*np.ones(3)
'''
## Three \bar{J} cases
'''
JI,JE,a,b=3.50,1.0,0.0,0.0
JEE,JIE,JEI,JII=JE+a,JE-a,JI-b,JI+b
'''
Am -- J(g0=0), l0(g0=0), r0(g0=0), S=R1(B1-lambda0In-1)^(-1)L1.T
'''
Am,Jsv=generate_meanmat_eig(Nparams,JEE,JIE,JEI,JII)
# calculate the original \bar{m} and \bar{n}
eigvAm,eigveclAm,eigvecrAm=scpla.eig(Am,left=True,right=True)
meigvecAm=eigvecrAm.copy()
neigvecAm=eigveclAm.copy()
for i in range(nrank):
    normalizecoe=(np.sum(neigvecAm[:,i]*meigvecAm[:,i]))
    neigvecAm[:,i]*=(eigvAm[i]/normalizecoe)
for i in range(nrank,NE*2):
    normalizecoe=(np.sum(neigvecAm[:,i]*meigvecAm[:,i]))
    neigvecAm[:,i]/=normalizecoe
print('normalization m & n')
print(la.norm(meigvecAm[:,0]),la.norm(neigvecAm[:,0]))
print('eigenvalues:',eigvAm[:nrank])
meanReigAmE,meanReigAmI=np.mean(meigvecAm[:NE,0]),np.mean(meigvecAm[NE:,0])
meanLeigAmE,meanLeigAmI=np.mean(neigvecAm[:NE,0])/eigvAm[0],np.mean(neigvecAm[NE:,0])/eigvAm[0]
''' for reference '''
x0,y0=np.reshape(meigvecAm[:,0],(NE*2,1)),np.reshape(neigvecAm[:,0].copy()/eigvAm[0],(NE*2,1))

''' Network Setting for Iterating Following '''
signeta=np.ones(3)
# signeta[1]*=(-1)
## heterogeneous random gain
''' >>>>>>>>>>>>>> Case 1 and Case 2 >>>>>>>>>>>>>>>>>>>>>>> '''
# xee,xei,xie,xii=1.0,1.0,1.0,1.0 ## HOMOgeneous random gain g_kl CASE 1
xee,xei,xie,xii=1.0,0.5,0.2,0.8 ## HETEROgeneous random gain CASE 2
# xee,xei,xie,xii=1.0,0.5,0.3,0.1
''' Recording Variables '''
# FOR m, n --> with lambda in 
eigvalseries,mvecseries,nvecseries=np.zeros((ngavg,ntrial,nrank),dtype=complex),np.zeros((ngavg,ntrial,NE*2,nrank),dtype=complex),np.zeros((ngavg,ntrial,NE*2,nrank),dtype=complex)
# for r,l --> without lambda 
Reigvecseries,Leigvecseries,Beigvseries=np.zeros((ngavg,ntrial,NE*2,N),dtype=complex),np.zeros((ngavg,ntrial,NE*2,N),dtype=complex),np.zeros((ngavg,ntrial,N),dtype=complex)

''' All recorded connectivity statistics '''
statesReig,statesLeig=np.zeros((ngavg,ntrial,3,2),dtype=complex),np.zeros((ngavg,ntrial,3,2),dtype=complex) #E,I,Total/mean,std# 
coveigvecseries=np.zeros((ngavg,ntrial,3),dtype=complex) # E, I and total
''' All random matrices for each trials '''
Xprimebasexee=np.zeros((ntrial,NE*2,NE*2)) 
# Y -- identical to X when g=1.0 ([-1])
''' Iterative Processing '''
gaverageseries=np.linspace(0.1,1.0,ngavg)
for iktrial in range(ntrial):
    '''
    for each individual trial, supposed to continuously increase g from zero to one -- see the sensibility changing
    but within one single trial, the random matrix (also random structure) keeps unchanged, thus avoid changing in random pattern
    '''
    # >>> generate the (base/ref) random matrix for each trial
    Xsym=np.reshape(np.random.randn(NE*2*NE*2),(NE*2,NE*2))/np.sqrt(NE)
    Xsym[:NE,:NE]=np.reshape(np.random.randn(NE*NE),(NE,NE))/np.sqrt(NE)
    Xsym[NE:,NE:]=np.reshape(np.random.randn(NE*NE),(NE,NE))/np.sqrt(NE)
    Xsym[NE:,:NE]=np.reshape(np.random.randn(NE*NE),(NE,NE))/np.sqrt(NE)
    Xsym[:NE,NE:]=np.reshape(np.random.randn(NE*NE),(NE,NE))/np.sqrt(NE)
    XsymT=Xsym.copy().T
    Yidendown=np.reshape(np.random.randn(NE*2*NE*2),(NE*2,NE*2))/np.sqrt(NE)
    Yidenup=np.reshape(np.random.randn(NE*2*NE*2),(NE*2,NE*2))/np.sqrt(NE)
    Xinit = Xsym.copy()
    # >>>>> Now, Xsym and XsymT keep the same while changing the random gain gaverage
    for idxeta,eta in enumerate(gaverageseries):
        #  >>> random gain continuously increases
        etaset=0.0*np.ones(3)#eta*np.ones(3)
        gaverage=eta        
        ''' heterogeneous ETA method 1 '''
        ## EE ##
        asqr=(1-np.sqrt(1-etaset[0]**2))/2.0 
        '''## aamp = 0, XT --> 0 X --> 1'''
        aamp=np.sqrt(asqr) 
        Xinit[:NE,:NE]=signeta[0]*aamp*XsymT[:NE,:NE].copy()+np.sqrt(1-aamp**2)*Xsym[:NE,:NE].copy()
        ## EI IE##
        asqr=(1-np.sqrt(1-etaset[1]**2))/2.0
        aamp=np.sqrt(asqr)
        Xinit[NE:,:NE]=signeta[1]*aamp*XsymT[NE:,:NE].copy()+np.sqrt(1-aamp**2)*Xsym[NE:,:NE].copy()
        Xinit[:NE,NE:]=signeta[1]*aamp*XsymT[:NE,NE:].copy()+np.sqrt(1-aamp**2)*Xsym[:NE,NE:].copy()
        ## II ##
        asqr=(1-np.sqrt(1-etaset[2]**2))/2.0
        aamp=np.sqrt(asqr)
        Xinit[NE:,NE:]=signeta[2]*aamp*XsymT[NE:,NE:].copy()+np.sqrt(1-aamp**2)*Xsym[NE:,NE:].copy()
        ''' THEORETICAL FULL ETA '''
        symtheofull=gaverage**2*(etaset[0]*signeta[0]+etaset[2]*signeta[2]+2*etaset[1]*signeta[1])/4.0#+(a-b)**2/(NE*2)
        # ''' heterogeneous '''
        X=Xinit.copy()
        X[:NE,:NE]*=(xee*gaverage/np.sqrt(2))
        X[:NE,NE:]*=(xei*gaverage/np.sqrt(2))
        X[NE:,:NE]*=(xie*gaverage/np.sqrt(2))
        X[NE:,NE:]*=(xii*gaverage/np.sqrt(2))

        '''
        >> start perturbed matrix
        >> * perturbed matrix: J; low rank structures: Am; random components: X
        >> * eigenvectors perturbed matrix: Reig, Leig --> renormalized by x0,y0,--> get xnorm0,ynorm0
        >> * low rank structure: x0,y0, 

        >>> statistics (recording)
        >> * mean and standard deviation of elements in eigenvectors -- E and I populations (individual)
        >> * overlap and tilting? between left and right eigenvectors --> relate to eigenvalue
        '''
        # overall
        J = X.copy()+Am.copy()
        JT = J.copy().T
        symnumfull=np.sum(np.sum(J*JT))/(4*NE**2)-np.mean(np.mean(J.copy()))**2
        symnumfull*=2*NE
        # print('eta full:',symtheofull,'; num:',symnumfull)
        '''   statistics >>>>>>>>>>> Full connectivity   '''
        # properties
        eigvJ,eigrvecJ=la.eig(J)
        inveigrvecJ=la.inv(eigrvecJ)
        meig = np.squeeze(eigrvecJ[:,:].copy())
        # meig = np.reshape(meig,(NE*2,nrank))
        neig = np.squeeze(inveigrvecJ[:,:].copy()) # inverse
        neig=neig.copy().T
        # neig = np.reshape(neig.T,(NE*2,nrank))
        for i in range(nrank):
            neig[:,i]*=eigvJ[i]
        for j in range(1,N):
            neig[:,j]*=eigvJ[j]
        
        Leig, Reig = np.zeros((N,N)),np.zeros((N,N))
        ''' The first '''
        Reig=meig.copy()
        normval=np.sum(Reig*neig.copy(),axis=0)
        normval=np.repeat(np.reshape(normval,(1,N)),N,axis=0)
        ''' eigendecomposition properties left and right eigenvectors '''
        Leig=neig.copy()/normval.copy() 
        '''  normalization 0 (reference)    '''
        tildex,tildey=np.reshape(Reig[:,0].copy(),(NE*2,1)),np.reshape(Leig[:,0].copy(),(NE*2,1))
        if (np.mean(tildey[:NE,0])*y0[0,0])<0:
            tildex*=(-1)
            tildey*=(-1)
        xnorm0=np.sqrt(np.squeeze(tildey.T@x0)/np.squeeze(y0.T@tildex))*tildex
        ynorm0=np.sqrt(np.squeeze(tildex.T@y0)/np.squeeze(x0.T@tildey))*tildey

        Reigvecseries[idxeta,iktrial,:,:],Leigvecseries[idxeta,iktrial,:,:]=Reig.copy(),Leig.copy()
        Beigvseries[idxeta,iktrial,:]=eigvJ.copy()
        Reigvecseries[idxeta,iktrial,:,0],Leigvecseries[idxeta,iktrial,:,0]=xnorm0[:,0].copy(),ynorm0[:,0].copy()
        ''' m n with eigenvalues in  '''
        ''' low rank properties m,n,eigenvalues '''
        mvecseries[idxeta,iktrial,:,0],nvecseries[idxeta,iktrial,:,0]=xnorm0[:,0].copy()*np.sqrt(NE*2),ynorm0[:,0].copy()*np.sqrt(NE*2)*eigvJ[0]

        '''  RECORDING STATES 
        * statesReig/Leig[grandom,trial,E/I/Total,mean/std]

        '''
        statesReig[idxeta,iktrial,0,0],statesReig[idxeta,iktrial,1,0]=np.mean(Reigvecseries[idxeta,iktrial,:NE,0]),np.mean(Reigvecseries[idxeta,iktrial,NE:,0])
        statesReig[idxeta,iktrial,2,0]=np.mean(Reigvecseries[idxeta,iktrial,:,0])
        statesReig[idxeta,iktrial,0,1],statesReig[idxeta,iktrial,1,1]=np.std(Reigvecseries[idxeta,iktrial,:NE,0]),np.std(Reigvecseries[idxeta,iktrial,NE:,0])
        statesReig[idxeta,iktrial,2,1]=np.std(Reigvecseries[idxeta,iktrial,:,0])
       
        statesLeig[idxeta,iktrial,0,0],statesLeig[idxeta,iktrial,1,0]=np.mean(Leigvecseries[idxeta,iktrial,:NE,0]),np.mean(Leigvecseries[idxeta,iktrial,NE:,0])
        statesLeig[idxeta,iktrial,2,0]=np.mean(Leigvecseries[idxeta,iktrial,:,0])
        statesLeig[idxeta,iktrial,0,1],statesLeig[idxeta,iktrial,1,1]=np.std(Leigvecseries[idxeta,iktrial,:NE,0]),np.std(Leigvecseries[idxeta,iktrial,NE:,0])
        statesLeig[idxeta,iktrial,2,1]=np.std(Leigvecseries[idxeta,iktrial,:,0])

        ''' covariance '''
        '''
        Note, we should change the normalization for the first item to the (Refer) normalization 
        tildeReig,tildeLeig
        normalization 0
        '''
        ## normalization 0 reference
        tildeReig,tildeLeig=np.squeeze(Reigvecseries[idxeta,iktrial,:,:]),np.squeeze(Leigvecseries[idxeta,iktrial,:,:])
        reig,leig=np.reshape(tildeReig[:,0].copy(),(NE*2,1)),np.reshape(tildeLeig[:,0].copy(),(NE*2,1))
        ## mean at eact point
        rEceneig,rIceneig = np.mean(reig[:NE,0]),np.mean(reig[NE:,0])
        lEceneig,lIceneig = np.mean(leig[:NE,0]),np.mean(leig[NE:,0])
        ## random components
        rrandeig,lrandeig=reig.copy(),leig.copy()
        rrandeig[:NE,0],rrandeig[NE:,0]=rrandeig[:NE,0]-rEceneig,rrandeig[NE:,0]-rIceneig
        lrandeig[:NE,0],lrandeig[NE:,0]=lrandeig[:NE,0]-lEceneig,lrandeig[NE:,0]-lIceneig

        coveigvecseries[idxeta,iktrial,0]=np.reshape(rrandeig[:NE,0],(NE,1)).T@np.reshape(lrandeig[:NE,0],(NE,1))
        coveigvecseries[idxeta,iktrial,1]=np.reshape(rrandeig[NE:,0],(NE,1)).T@np.reshape(lrandeig[NE:,0],(NE,1))
        coveigvecseries[idxeta,iktrial,2]=coveigvecseries[idxeta,iktrial,0]+coveigvecseries[idxeta,iktrial,1]

        ''' record the random derivative '''
        if idxeta == ngavg-1:
            print('recording!!!')
            Xprimebasexee[iktrial,:,:]=X.copy()


* recording data

In [ ]:
# saving data
now = time.strftime("%Y-%m-%d-%H_%M_%S",time.localtime(time.time()))
Jparameters = np.zeros(4+1)
Jparameters[0],Jparameters[1],Jparameters[2],Jparameters[3]=JEE,JEI,JIE,JII
Jparameters[4]=NE
Randgain=np.zeros(4)
Randgain[0],Randgain[1],Randgain[2],Randgain[3]=xee,xei,xie,xii

np.savez(now+'_perturbationHeterZeroeta_data',x0=x0,y0=y0,Jparameters=Jparameters,Randgain=Randgain,gaverageseries=gaverageseries,etaset=etaset,signeta=signeta,eigvalseries=eigvalseries,Reigvecseries=Reigvecseries[:,:,:,:nrank],Leigvecseries=Leigvecseries[:,:,:,:nrank],Beigvseries=Beigvseries[:,:,:nrank],meigvecAm=meigvecAm[:,:nrank],neigvecAm=neigvecAm[:,:nrank],eigvAm=eigvAm,statesReig=statesReig,statesLeig=statesLeig,coveigvecseries=coveigvecseries,Xprimebasexee=Xprimebasexee)

* eigenvalue spectrum

In [ ]:
cm='bgc'
idxc=0
for idxgavg in range(ngavg):
    figtspt,axtspt=plt.subplots(figsize=(6,3))
    for idxtrial in range(ntrial):
        axtspt.scatter(np.real(Beigvseries[idxgavg,idxtrial,nrank:]),np.imag(Beigvseries[idxgavg,idxtrial,nrank:]),s=5,c=cm[idxc],alpha=0.25) # >>>>>>>>>>>>>>
    axtspt.scatter(np.real(Beigvseries[idxgavg,:,0]),np.imag(Beigvseries[idxgavg,:,0])-0.2*idxgavg,s=20,c='red',marker='^',alpha=0.5) # >>>>>>>>>>>>
    axtspt.set_aspect('equal')
    axtspt.plot(np.ones(2)*eigvAm[0],np.array([-1.0,1.0]),linestyle='--',linewidth=1.5,color='grey')
    axtspt.spines['right'].set_color('none')
    axtspt.spines['top'].set_color('none')
    axtspt.xaxis.set_ticks_position('bottom')
    axtspt.spines['bottom'].set_position(('data', 0))
    xticks = np.linspace(-3.5,2.5,2)
    xlims = [-3.5,2.5]
    yticks = np.linspace(-2.0,2.0,2)
    ylims = [-2.0,2.0]
    axtspt.set_xlim(xlims)
    axtspt.set_ylim(ylims)
    axtspt.set_xticks(xticks)
    axtspt.set_yticks(yticks)
    axtspt.set_aspect('equal')

In [ ]:

'''
with symmetry, subcircuit
eigvJ0
'''
idxgavg,idxtrial=9,3 # 6,3,0
figtspt,axtspt=plt.subplots(figsize=(6,3))
cm='bgc'
idxc=0
axtspt.scatter(np.real(Beigvseries[idxgavg,idxtrial,nrank:]),np.imag(Beigvseries[idxgavg,idxtrial,nrank:]),s=5,c=cm[idxc],alpha=0.25) # >>>>>>>>>>>>>>
axtspt.scatter(np.real(Beigvseries[idxgavg,:,0]),np.imag(Beigvseries[idxgavg,:,0])-0.2*idxc,s=20,c=cm[idxc],marker='^',alpha=0.5) # >>>>>>>>>>>>
axtspt.set_aspect('equal')
axtspt.plot(np.ones(2)*eigvAm[0],np.array([-1.0,1.0]),linestyle='--',linewidth=1.5,color='grey')
axtspt.spines['right'].set_color('none')
axtspt.spines['top'].set_color('none')
axtspt.xaxis.set_ticks_position('bottom')
axtspt.spines['bottom'].set_position(('data', 0))

aee,aei,aie,aii=xee,xei,xie,xii
print(xee,xei,xie,xii)
idxtrial=10
eta=0
theta = np.linspace(0, 2 * np.pi, 200)
# first do not multiply at
ahomo=gaverageseries[idxgavg]
xee_,xei_,xie_,xii_=ahomo*aee/np.sqrt(2),ahomo*aei/np.sqrt(2),ahomo*aie/np.sqrt(2),ahomo*aii/np.sqrt(2)
gmat = np.array([[xee_**2,xei_**2],[xie_**2,xii_**2]])
gaverage_=0
for i in range(2):
    for j in range(2):
        gaverage_+=gmat[i,j]/2 # ntype=2
gaverage_=np.sqrt(gaverage_)
eigvgm,eigvecgm=la.eig(gmat) 
r_g2=np.max(eigvgm)
r_g = np.sqrt(r_g2)
print(r_g)
eta=0
longaxis,shortaxis=(1+eta)*r_g ,(1-eta)*r_g 
xr = longaxis*np.cos(theta)
yr = shortaxis*np.sin(theta)
axtspt.plot(xr, yr, color="gray", linewidth=2.5,linestyle='--',label=r'ellipse') # >>>>>
axtspt.plot(-longaxis*np.ones(2), np.array([-1.0,1.0]), color="gray", linewidth=2.5,linestyle='--') # >>>>>
axtspt.set_title(r'$\eta=$'+np.str(eta))
xticks = np.linspace(-2.5,3.5,2)
xlims = [-3.5,3.5]
yticks = np.linspace(-1.0,1.0,2)
ylims = [-1.1,1.1]
# axtspt.set_xlim(xlims)
# axtspt.set_ylim(ylims)
# axtspt.set_xticks(xticks)
# axtspt.set_yticks(yticks)
axtspt.set_aspect('equal')

* loading patterns

In [ ]:
'''
loading patterns
for ensemble-averaging results
>>>>>>>>>>>>>>>>> with eigenvalues >>>>>>>>>>>>>>>
'''
xticks = np.linspace(-0.4,0.45,2)
xlims = [-0.4,0.45]
yticks = np.linspace(-0.4,0.45,2)
ylims = [-0.4,0.45]
# xticks = np.linspace(-0.2,0.25,2)
# xlims = [-0.2,0.25]
# yticks = np.linspace(-0.2,0.25,2)
# ylims = [-0.2,0.25]
xticks = np.linspace(-0.15,0.20,2)
xlims = [-0.15,0.20]
yticks = np.linspace(-0.05,0.10,2)
ylims = [-0.05,0.1]
theta = np.linspace(0, 2 * np.pi, 200)
''' how eigenvalue outlier changes with g_bar '''

idxtrial=9#16#
idxeta=3#9# 
alphaval=0.10
edgv='black'
cm='br'
# fig,ax=plt.subplots(figsize=(5,3))  
nrank=1
ntrial = np.shape(Reigvecseries)[1]
NE=int(NE)
print(NE)
'''loading vector changing'''
meanmE,meanmI,meannE,meannI=np.zeros(nrank),np.zeros(nrank),np.zeros(nrank),np.zeros(nrank)

mEvec,mIvec,nEvec,nIvec=np.squeeze(Reigvecseries[idxeta,:,:NE,0]),np.squeeze(Reigvecseries[idxeta,:,NE:,0]),np.squeeze(Leigvecseries[idxeta,:,:NE,0]),np.squeeze(Leigvecseries[idxeta,:,NE:,0])
mEvec,mIvec,nEvec,nIvec=mEvec.flatten(),mIvec.flatten(),nEvec.flatten(),nIvec.flatten()
scale_std=3.0
for irank in range(nrank):
    meanmEtotal,stdmEtotal = np.mean(mEvec),np.std(mEvec)
    varmE = mEvec - meanmEtotal
    idxwhere = np.where(np.abs(varmE)>scale_std*stdmEtotal)
    mEvec[idxwhere]=meanmEtotal
    meanmE[irank]=np.mean(mEvec)
    # puring I
    meanmItotal,stdmItotal = np.mean(mIvec),np.std(mIvec)
    varmI = mIvec - meanmItotal
    idxwhere = np.where(np.abs(varmI)>scale_std*stdmItotal)
    mIvec[idxwhere]=meanmItotal
    meanmI[irank]=np.mean(mIvec)
    
    # n vector
    # puring E
    meannEtotal,stdnEtotal = np.mean(nEvec),np.std(nEvec)
    varnE = nEvec - meannEtotal
    idxwhere = np.where(np.abs(varnE)>scale_std*stdnEtotal)
    nEvec[idxwhere]=meannEtotal
    meannE[irank]=np.mean(nEvec)

    # puring I
    meannItotal,stdnItotal = np.mean(nIvec),np.std(nIvec)
    varnI = nIvec - meannItotal
    idxwhere = np.where(np.abs(varnI)>scale_std*stdnItotal)
    nIvec[idxwhere]=meannItotal
    meannI[irank]=np.mean(nIvec)

''' calculate directions '''
noiseE,noiseI = np.zeros((NE*ntrial,2,nrank)),np.zeros((NE*ntrial,2,nrank))
dirvecE,dirvecI=np.zeros((2,2)),np.zeros((2,2))
for irank in range(nrank):
    # E 0 M 1 N
    noiseE[:,0,irank],noiseE[:,1,irank]= nEvec-meannE[irank],mEvec-meanmE[irank]
    noiseI[:,0,irank],noiseI[:,1,irank]= nIvec-meannI[irank],mIvec-meanmI[irank]
# m1n1
covdirE,covdirI=np.squeeze(noiseE[:,:,0]).T@np.squeeze(noiseE[:,:,0]),np.squeeze(noiseI[:,:,0]).T@np.squeeze(noiseI[:,:,0])
_,dirvecE=la.eig(covdirE)
_,dirvecI=la.eig(covdirI)  

ax.scatter(nIvec*np.sqrt(N)*eigvAm[0],mIvec*np.sqrt(N),s=5.0,c='green',alpha=alphaval)#cm[1],alpha=alphaval)
ax.scatter(nEvec*np.sqrt(N)*eigvAm[0],mEvec*np.sqrt(N),s=5.0,c='r',alpha=alphaval)#cm[0],alpha=alphaval)

ax.plot([meannE[0],meannE[0]+dirvecE[0,0]],[meanmE[0],meanmE[0]+dirvecE[1,0]],color=edgv,linestyle='--',linewidth=1.5)
ax.plot([meannE[0],meannE[0]+dirvecE[0,1]],[meanmE[0],meanmE[0]+dirvecE[1,1]],color=edgv,linestyle='--',linewidth=1.5)
ax.plot([meannI[0],meannI[0]+dirvecI[0,0]],[meanmI[0],meanmI[0]+dirvecI[1,0]],color=edgv,linestyle='--',linewidth=1.5)
ax.plot([meannI[0],meannI[0]+dirvecI[0,1]],[meanmI[0],meanmI[0]+dirvecI[1,1]],color=edgv,linestyle='--',linewidth=1.5)

confidence_ellipse(np.real(nEvec),np.real(mEvec),ax,edgecolor=edgv)
confidence_ellipse(np.real(nIvec),np.real(mIvec),ax,edgecolor=edgv)
ax.axvline(c='grey', lw=1)
ax.axhline(c='grey', lw=1)
# ax.set_xlim(xlims)
# ax.set_ylim(ylims)
# ax.set_xticks(xticks)
# ax.set_yticks(yticks)
ax.set_aspect('equal')


In [ ]:
'''
m n
>>>>>>>>>>>>>>>>> with eigenvalues >>>>>>>>>>>>>>>
'''
xticks = np.linspace(-15,10,2)
xlims = [-15,10]
yticks = np.linspace(-1,4,2)
ylims = [-1,4]

theta = np.linspace(0, 2 * np.pi, 200)
''' how eigenvalue outlier changes with g_bar '''

idxtrial=9#16#
idxeta=9#9# 
alphaval=0.10
edgv='black'
cm='br'
fig,ax=plt.subplots(figsize=(5,3))  
nrank=1
ntrial = np.shape(Reigvecseries)[1]
NE=int(NE)
print(NE)
'''loading vector changing'''
meanmE,meanmI,meannE,meannI=np.zeros(nrank),np.zeros(nrank),np.zeros(nrank),np.zeros(nrank)

mEvec,mIvec,nEvec,nIvec=np.squeeze(mvecseries[idxeta,:,:NE,0]),np.squeeze(mvecseries[idxeta,:,NE:,0]),np.squeeze(nvecseries[idxeta,:,:NE,0]),np.squeeze(nvecseries[idxeta,:,NE:,0])
mEvec,mIvec,nEvec,nIvec=mEvec.flatten(),mIvec.flatten(),nEvec.flatten(),nIvec.flatten()
scale_std=3.0
for irank in range(nrank):
    meanmEtotal,stdmEtotal = np.mean(mEvec),np.std(mEvec)
    varmE = mEvec - meanmEtotal
    idxwhere = np.where(np.abs(varmE)>scale_std*stdmEtotal)
    mEvec[idxwhere]=meanmEtotal
    meanmE[irank]=np.mean(mEvec)
    # puring I
    meanmItotal,stdmItotal = np.mean(mIvec),np.std(mIvec)
    varmI = mIvec - meanmItotal
    idxwhere = np.where(np.abs(varmI)>scale_std*stdmItotal)
    mIvec[idxwhere]=meanmItotal
    meanmI[irank]=np.mean(mIvec)
    
    # n vector
    # puring E
    meannEtotal,stdnEtotal = np.mean(nEvec),np.std(nEvec)
    varnE = nEvec - meannEtotal
    idxwhere = np.where(np.abs(varnE)>scale_std*stdnEtotal)
    nEvec[idxwhere]=meannEtotal
    meannE[irank]=np.mean(nEvec)

    # puring I
    meannItotal,stdnItotal = np.mean(nIvec),np.std(nIvec)
    varnI = nIvec - meannItotal
    idxwhere = np.where(np.abs(varnI)>scale_std*stdnItotal)
    nIvec[idxwhere]=meannItotal
    meannI[irank]=np.mean(nIvec)

''' calculate directions '''
noiseE,noiseI = np.zeros((NE*ntrial,2,nrank)),np.zeros((NE*ntrial,2,nrank))
dirvecE,dirvecI=np.zeros((2,2)),np.zeros((2,2))
for irank in range(nrank):
    # E 0 M 1 N
    noiseE[:,0,irank],noiseE[:,1,irank]= nEvec-meannE[irank],mEvec-meanmE[irank]
    noiseI[:,0,irank],noiseI[:,1,irank]= nIvec-meannI[irank],mIvec-meanmI[irank]
# m1n1
covdirE,covdirI=np.squeeze(noiseE[:,:,0]).T@np.squeeze(noiseE[:,:,0]),np.squeeze(noiseI[:,:,0]).T@np.squeeze(noiseI[:,:,0])
_,dirvecE=la.eig(covdirE)
_,dirvecI=la.eig(covdirI) 
for i in range(2):
    for j in range(2):
        dirvecE[i,j]*=(1*np.sqrt(N))
        dirvecI[i,j]*=(1*np.sqrt(N))

ax.scatter(nIvec,mIvec,s=5.0,c='blue',alpha=alphaval)#cm[1],alpha=alphaval)
ax.scatter(nEvec,mEvec,s=5.0,c='red',alpha=alphaval)#cm[0],alpha=alphaval)

ax.plot([meannE[0],meannE[0]+dirvecE[0,0]],[meanmE[0],meanmE[0]+dirvecE[1,0]],color=edgv,linestyle='--',linewidth=1.5)
ax.plot([meannE[0],meannE[0]+dirvecE[0,1]],[meanmE[0],meanmE[0]+dirvecE[1,1]],color=edgv,linestyle='--',linewidth=1.5)
ax.plot([meannI[0],meannI[0]+dirvecI[0,0]],[meanmI[0],meanmI[0]+dirvecI[1,0]],color=edgv,linestyle='--',linewidth=1.5)
ax.plot([meannI[0],meannI[0]+dirvecI[0,1]],[meanmI[0],meanmI[0]+dirvecI[1,1]],color=edgv,linestyle='--',linewidth=1.5)

confidence_ellipse(np.real(nEvec),np.real(mEvec),ax,edgecolor=edgv)
confidence_ellipse(np.real(nIvec),np.real(mIvec),ax,edgecolor=edgv)
ax.axvline(c='grey', lw=1)
ax.axhline(c='grey', lw=1)
ax.set_xlim(xlims)
ax.set_ylim(ylims)
ax.set_xticks(xticks)
ax.set_yticks(yticks)
ax.set_aspect('equal')


## individual perturbations of left and right eigenvectors
* calculate the predicted perturbations (theo)
* comparing with numerical perturbations (num)

In [ ]:
'''
>>>>>>>>>>>>>>>>> Compare the individual response (not statistics), NOTE:
To make direct comparison between numerical and perturbation theorem, keep the same random matrix for each trial
choose one trial -- determine the random derivative -- calculate the individual elements in vector

'''

'''
# loading data
# strfile='2021-03-13-15_20_04_perturbationHomoZeroeta_data'# homogeneous random gain
strfile='2021-03-13-15_10_37_perturbationHeterZeroeta_data' # heterogeneous random gain
data = np.load(strfile+'.npz')
gaverageseries = data['gaverageseries']
x0,y0=data['x0'],data['y0']
meigvecAm,neigvecAm=data['meigvecAm'],data['neigvecAm']
# perturbed
Leigvecseries,Reigvecseries=data['Leigvecseries'],data['Reigvecseries']
Xprimebasexee=data['Xprimebasexee']
eigvAm=data['eigvAm']
ngavg,ntrial,N=np.shape(Leigvecseries)
Leigvecseries=np.expand_dims(Leigvecseries,axis=3)
Reigvecseries=np.expand_dims(Reigvecseries,axis=3)
NE=int(N/2.0)
'''

'''
calculate S0
'''
xticks = np.linspace(-0.08,0.08,2)
xlims = [-0.08,0.08]
yticks = np.linspace(-0.08,0.08,2)
ylims = [-0.08,0.08]
Z = np.zeros((NE*2-1,NE*2-1))
for i in range(NE*2-1):
    b1=eigvAm[i+1]
    Z[i,i]=1/(-eigvAm[0])
S0=np.zeros((NE*2,NE*2))
R1,L1=np.squeeze(meigvecAm[:,1:]),np.squeeze(neigvecAm[:,1:])
S0=R1@Z@L1.T

'''
# CHOOSE ONE TRIAL
'''
idxtrial,idxtrial_=8,3
fig,ax=plt.subplots(2,3,figsize=(6,4))
Xprime = np.squeeze(Xprimebasexee[idxtrial,:,:])
idxgsamples=np.array([1,5,9])
gsamples=gaverageseries[idxgsamples]
rperturb_num,lperturb_num = np.zeros((NE*2,3)),np.zeros((NE*2,3))
rperturb_the,lperturb_the = np.zeros((NE*2,3)),np.zeros((NE*2,3))
rprime_the,lprime_the = np.squeeze(Xprime@x0/eigvAm[0]),np.squeeze(Xprime.T@y0/eigvAm[0])
for i in range(len(idxgsamples)):
    lperturb_num[:,i] = np.squeeze(Leigvecseries[idxgsamples[i],idxtrial,:,0])-np.squeeze(y0)
    rperturb_num[:,i] = np.squeeze(Reigvecseries[idxgsamples[i],idxtrial,:,0])-np.squeeze(x0)

    lperturb_the[:,i] = lprime_the*gsamples[i]
    rperturb_the[:,i] = rprime_the*gsamples[i]
    ax[0,i].plot(xticks,yticks,color='darkred',linestyle='--')
    ax[1,i].plot(xticks,yticks,color='darkred',linestyle='--')
    ax[0,i].scatter(np.real(rperturb_the[:,i]),rperturb_num[:,i],s=5,c='gray',alpha=0.5)
    ax[1,i].scatter(np.real(lperturb_the[:,i]),lperturb_num[:,i],s=5,c='gray',alpha=0.5)
    ax[0,i].set_xticks([])
    ax[0,i].set_yticks([])
    ax[1,i].set_xticks([])
    ax[1,i].set_yticks([])

for i in range(3):
    ax[0,i].set_xlim(xlims)
    ax[0,i].set_ylim(ylims)
    ax[1,i].set_xlim(xlims)
    ax[1,i].set_ylim(ylims)
ax[0,0].set_yticks(yticks)
ax[1,0].set_yticks(yticks)
for i in range(3):
    ax[1,i].set_xticks(xticks)
# print(np.shape(rprime_the),np.shape(rprime_num))
# ax.scatter(np.real(rprime_the),rprime_num,c='gray',alpha=0.5)
# ax.set_xlim([-0.05,0.05])
# ax.set_ylim([-0.05,0.05])
'''
# CHOOSE TWO TRIALS
'''
idxtrial,idxtrial_=8,3
fig,ax=plt.subplots(2,3,figsize=(6,4))
Xprime = np.squeeze(Xprimebasexee[idxtrial,:,:])
idxgsamples=np.array([1,5,9])
gsamples=gaverageseries[idxgsamples]
rperturb_num,lperturb_num = np.zeros((NE*2,3)),np.zeros((NE*2,3))
rperturb_the,lperturb_the = np.zeros((NE*2,3)),np.zeros((NE*2,3))
rprime_the,lprime_the = np.squeeze(Xprime@x0/eigvAm[0]),np.squeeze(Xprime.T@y0/eigvAm[0])
for i in range(len(idxgsamples)):
    lperturb_num[:,i] = np.squeeze(Leigvecseries[idxgsamples[i],idxtrial_,:,0])-np.squeeze(y0)
    rperturb_num[:,i] = np.squeeze(Reigvecseries[idxgsamples[i],idxtrial_,:,0])-np.squeeze(x0)

    lperturb_the[:,i] = lprime_the*gsamples[i]
    rperturb_the[:,i] = rprime_the*gsamples[i]
    ax[0,i].plot(xticks,yticks,color='darkred',linestyle='--')
    ax[1,i].plot(xticks,yticks,color='darkred',linestyle='--')
    ax[0,i].scatter(np.real(rperturb_the[:,i]),rperturb_num[:,i],s=5,c='gray',alpha=0.5)
    ax[1,i].scatter(np.real(lperturb_the[:,i]),lperturb_num[:,i],s=5,c='gray',alpha=0.5)
    ax[0,i].set_xticks([])
    ax[0,i].set_yticks([])
    ax[1,i].set_xticks([])
    ax[1,i].set_yticks([])



for i in range(3):
    ax[0,i].set_xlim(xlims)
    ax[0,i].set_ylim(ylims)
    ax[1,i].set_xlim(xlims)
    ax[1,i].set_ylim(ylims)
ax[0,0].set_yticks(yticks)
ax[1,0].set_yticks(yticks)
for i in range(3):
    ax[1,i].set_xticks(xticks)
# print(np.shape(rprime_the),np.shape(rprime_num))
# ax.scatter(np.real(rprime_the),rprime_num,c='gray',alpha=0.5)
# ax.set_xlim([-0.05,0.05])
# ax.set_ylim([-0.05,0.05])

## population statistics 
* first determint lemma for eigenvalue outlier -- no change
* perturbation theorem for variance of eigenvalue
* perturbation theorem for variance of left and right eigenvectors


In [ ]:
'''
### >>>>>>>>>>>>>>>>> Statistics Linearly Change >>>>>>>>>>>>>>>>
Case 1,2 focus on second-order statstics WITHIN (left/right) eigenvectors
Calculate the statistics of eigenvalue, eigenvectors
  * mean, variance of the perturbation of eigenvalue \mu(derivative of lambda),\sigma(derivative of lambda)
'''
'''
Variables:
    std_perturb_Lambda_num (1)
    std_perturb_Lambda_theo (1)
    std_perturbLambda_linear: reconstruct np.zeros(ngavg)
    std_perturbLambda_linear_theo=std_perturb_Lambda_theo*gaverageseries
'''
## calculating the statistics of perturbation lambda
# Coding uses only one point [linear assumption]
std_perturb_Lambda_numSE  = np.std(eigvalseries[ngavg-1,:,0])
std_perturb_Lambda_theo = np.sqrt((JE**2*(xee**2+xei**2)+JI**2*(xie**2+xii**2))/(2*NE)/(JE-JI)**2)
## theoretical curve for standard deviation of the eigenvalue
std_perturbLambda_linear_theo=std_perturb_Lambda_theo*gaverageseries
std_perturbLambda_linear_numSE=std_perturb_Lambda_numSE*gaverageseries
'''
Variance of the perturbations in eigenvalue lambda
\delta lambda =\Delta lambda = (1) unit change in g
'''
std_perturb_Lambda_num = np.std(Beigvseries[:,:,0],axis=1)
figE,axE=plt.subplots(figsize=(4,4))
cm='rbg'
for i in range(ntrial):
    axE.scatter(gaverageseries,np.real(Beigvseries[:,i,0])-eigvAm[0],s=10,color='darkblue',marker='^',label='numerical')
# axE.plot(gaverageseries,std_perturb_Lambda_num,color=cm[1],linewidth=1.5,label='numerical')
# polyfitting
p=np.polyfit(gaverageseries,std_perturb_Lambda_num,1)
std_perturbLambda_linear_num = np.zeros(ngavg)
std_perturbLambda_linear_num = p[0]*gaverageseries+p[1]
# axE.plot(gaverageseries,std_perturbLambda_linear,color=cm[2],linestyle='--',linewidth=1.5,alpha=0.5,label='1-order polynomial')
axE.plot(gaverageseries,std_perturbLambda_linear_theo,color='darkgreen',linewidth=1.5,label='analytical')
print('std delta_lambda numerical:',p[0])
axE.set_title(r'$\sigma_{\lambda}$')

'''
plt and ax
'''
ylims=[-0.08,0.08]
xlims=[-0.0,1.1]
yticks = np.linspace(-0.08,0.08,2)
xticks = np.linspace(0.0,1.0,2)
axE.set_xlim(xlims)
axE.set_xticks(xticks)
axE.set_ylim(ylims)
axE.set_yticks(yticks)
# ax.legend()

'''
right eigenvector
'''
## calculating the statistics of perturbation lambda
# Coding uses only one point
std_perturb_Reig_numSE,std_perturb_Reig_theo=np.zeros(2),np.zeros(2)
std_perturb_Reig_numSE[0],std_perturb_Reig_numSE[1] = np.mean(statesReig[ngavg-1,:,0,1]),np.mean(statesReig[ngavg-1,:,1,1])
std_perturb_Reig_theo[0],std_perturb_Reig_theo[1] = np.sqrt(((xee)**2+(xei)**2)/(2*2*NE)/eigvAm[0]**2),np.sqrt(((xie)**2+(xii)**2)/(2*2*NE)/eigvAm[0]**2)
## theoretical curve for standard deviation of the eigenvalue
std_perturbReig_linear_theo,std_perturbReig_linear_numSE=np.zeros((ngavg,2)),np.zeros((ngavg,2))
std_perturbReig_linear_theo[:,0]=std_perturb_Reig_theo[0]*gaverageseries
std_perturbReig_linear_theo[:,1]=std_perturb_Reig_theo[1]*gaverageseries
std_perturbReig_linear_numSE[:,0]=std_perturb_Reig_numSE[0]*gaverageseries
std_perturbReig_linear_numSE[:,1]=std_perturb_Reig_numSE[1]*gaverageseries

print('std of perturbation right eigenvector, num:',std_perturb_Reig_numSE,'; theo:',std_perturb_Reig_theo)
'''
Variance of the perturbations in eigenvalue lambda
\delta lambda =\Delta lambda = (1) unit change in g
'''
std_perturb_Reig_num,std_perturbReig_linear_num=np.zeros((ngavg,2)),np.zeros((ngavg,2))
std_perturb_Reig_num[:,0],std_perturb_Reig_num[:,1] = np.mean(statesReig[:,:,0,1],axis=1),np.mean(statesReig[:,:,1,1],axis=1)

figR0,axR0=plt.subplots(2,1,figsize=(4,4))
cm='rbg'
# >>>>>>>> numerical solution
axR0[0].scatter(gaverageseries,std_perturb_Reig_num[:,0],color='darkred',label='numerical')
axR0[1].scatter(gaverageseries,std_perturb_Reig_num[:,1],color='darkblue',label='numerical')
axR0[0].scatter(gaverageseries,std_perturbReig_linear_numSE[:,0],color='orange',label='numerical')
axR0[1].scatter(gaverageseries,std_perturbReig_linear_numSE[:,1],color='cyan',label='numerical')
# >>>>>>> Theoretical solutions
axR0[0].plot(gaverageseries,std_perturbReig_linear_theo[:,0],color='red',linewidth=1.5,label='analytical')
axR0[1].plot(gaverageseries,std_perturbReig_linear_theo[:,1],color='blue',linewidth=1.5,label='analytical')
axR0[0].set_title(r'$\sigma_{r}^E$')
axR0[1].set_title(r'$\sigma_{r}^I$')

figR,axR=plt.subplots(2,1,figsize=(4,4))
cm='rbg'
# >>>>>>>> numerical solution
for i in range(ntrial):
    axR[0].scatter(gaverageseries,statesReig[:,i,0,1],s=10,c='darkred',marker='^',label='numerical')
    axR[1].scatter(gaverageseries,statesReig[:,i,1,1],s=10,c='darkblue',marker='^',label='numerical')
# >>>>>>> Theoretical solutions
axR[0].plot(gaverageseries,std_perturbReig_linear_theo[:,0],color='red',linewidth=1.5,label='analytical')
axR[1].plot(gaverageseries,std_perturbReig_linear_theo[:,1],color='blue',linewidth=1.5,label='analytical')
axR[0].set_title(r'$\sigma_{r}^E$')
axR[1].set_title(r'$\sigma_{r}^I$')

ylims=[-0.00,0.015]
xlims=[-0.0,1.1]
yticks = np.linspace(0.0,0.015,2)
xticks = np.linspace(0.0,1.0,2)
for i in range(2):
    axR[i].set_xlim(xlims)
    axR[i].set_xticks([])
    axR[i].set_ylim(ylims)
    axR[i].set_yticks(yticks)

    axR0[i].set_xlim(xlims)
    axR0[i].set_xticks([])
    axR0[i].set_ylim(ylims)
    axR0[i].set_yticks(yticks)
axR[1].set_xticks(xticks)
axR0[1].set_xticks(xticks)

'''
left eigenvector
'''
## calculating the statistics of perturbation lambda
# Coding uses only one point
std_perturb_Leig_numSE,std_perturb_Leig_theo=np.zeros(2),np.zeros(2)
std_perturb_Leig_numSE[0],std_perturb_Leig_numSE[1] = np.mean(statesLeig[ngavg-1,:,0,1]),np.mean(statesLeig[ngavg-1,:,1,1])
std_perturb_Leig_theo[0],std_perturb_Leig_theo[1] = np.sqrt(2*((JE*xee)**2+(JI*xie)**2)/(2*NE)/(JE-JI)**2/eigvAm[0]**2),np.sqrt(2*((JE*xei)**2+(JI*xii)**2)/(2*NE)/(JE-JI)**2/eigvAm[0]**2)
## theoretical curve for standard deviation of the eigenvalue
std_perturbLeig_linear_theo,std_perturbLeig_linear_numSE=np.zeros((ngavg,2)),np.zeros((ngavg,2))
std_perturbLeig_linear_theo[:,0]=std_perturb_Leig_theo[0]*gaverageseries
std_perturbLeig_linear_theo[:,1]=std_perturb_Leig_theo[1]*gaverageseries
std_perturbLeig_linear_numSE[:,0]=std_perturb_Leig_numSE[0]*gaverageseries
std_perturbLeig_linear_numSE[:,1]=std_perturb_Leig_numSE[1]*gaverageseries

print('std of perturbation left eigenvector, num:',std_perturb_Leig_numSE,'; theo:',std_perturb_Leig_theo)
'''
Variance of the perturbations in eigenvalue lambda
\delta lambda =\Delta lambda = (1) unit change in g
'''
std_perturb_Leig_num,std_perturbLeig_linear_num=np.zeros((ngavg,2)),np.zeros((ngavg,2))
std_perturb_Leig_num[:,0],std_perturb_Leig_num[:,1] = np.mean(statesLeig[:,:,0,1],axis=1),np.mean(statesLeig[:,:,1,1],axis=1)

figL0,axL0=plt.subplots(2,1,figsize=(4,4))
cm='rbg'
# >>>>>>>> numerical solution
axL0[0].scatter(gaverageseries,std_perturb_Leig_num[:,0],color='darkred',label='numerical')
axL0[1].scatter(gaverageseries,std_perturb_Leig_num[:,1],color='darkblue',label='numerical')
axL0[0].scatter(gaverageseries,std_perturbLeig_linear_numSE[:,0],color='orange',label='numerical')
axL0[1].scatter(gaverageseries,std_perturbLeig_linear_numSE[:,1],color='cyan',label='numerical')
# >>>>>>> Theoretical solutions
axL0[0].plot(gaverageseries,std_perturbLeig_linear_theo[:,0],color='red',linewidth=1.5,label='analytical')
axL0[1].plot(gaverageseries,std_perturbLeig_linear_theo[:,1],color='blue',linewidth=1.5,label='analytical')
axL0[0].set_title(r'$\sigma_{r}^E$')
axL0[1].set_title(r'$\sigma_{r}^I$')

figL,axL=plt.subplots(2,1,figsize=(4,4))
cm='mg'#'rb'#
# >>>>>>>> numerical solution
for i in range(ntrial):
    axL[0].scatter(gaverageseries,statesLeig[:,i,0,1],s=10,c='darkred',marker='^',label='numerical')
    axL[1].scatter(gaverageseries,statesLeig[:,i,1,1],s=10,c='darkblue',marker='^',label='numerical')
# >>>>>>> Theoretical solutions
axL[0].plot(gaverageseries,std_perturbLeig_linear_theo[:,0],color='red',linewidth=1.5,label='analytical')
axL[1].plot(gaverageseries,std_perturbLeig_linear_theo[:,1],color='blue',linewidth=1.5,label='analytical')
axL[0].set_title(r'$\sigma_{l}^E$')
axL[1].set_title(r'$\sigma_{l}^I$')

ylims=[-0.00,0.025]
xlims=[-0.0,1.1]
yticks = np.linspace(0.0,0.025,2)
xticks = np.linspace(0.0,1.0,2)
for i in range(2):
    axL[i].set_xlim(xlims)
    axL[i].set_xticks([])
    axL[i].set_ylim(ylims)
    axL[i].set_yticks(yticks)

    axL0[i].set_xlim(xlims)
    axL0[i].set_xticks([])
    axL0[i].set_ylim(ylims)
    axL0[i].set_yticks(yticks)
axL[1].set_xticks(xticks)
axL0[1].set_xticks(xticks)

In [ ]:
'''  m, n '''
## calculating the statistics of perturbation lambda
# Coding uses only one point [linear assumption]
std_perturb_Lambda_numSE  = np.std(eigvalseries[ngavg-1,:,0])
std_perturb_Lambda_theo = np.sqrt((JE**2*(xee**2+xei**2)+JI**2*(xie**2+xii**2))/(2*NE)/(JE-JI)**2)
## theoretical curve for standard deviation of the eigenvalue
std_perturbLambda_linear_theo=std_perturb_Lambda_theo*gaverageseries
std_perturbLambda_linear_numSE=std_perturb_Lambda_numSE*gaverageseries
'''
Variance of the perturbations in eigenvalue lambda
\delta lambda =\Delta lambda = (1) unit change in g
'''
std_perturb_Lambda_num = np.std(Beigvseries[:,:,0],axis=1)
figE,axE=plt.subplots(figsize=(4,4))
cm='rbg'
for i in range(ntrial):
    axE.scatter(gaverageseries,np.real(Beigvseries[:,i,0])-eigvAm[0],s=15,color='gray',marker='^',label='numerical')
axE.plot(gaverageseries,std_perturb_Lambda_num,color=cm[1],linewidth=1.5,linestyle='--',label='numerical')
axE.plot(gaverageseries,-std_perturb_Lambda_num,color=cm[1],linewidth=1.5,linestyle='--',label='numerical')
# polyfitting
p=np.polyfit(gaverageseries,std_perturb_Lambda_num,1)
std_perturbLambda_linear_num = np.zeros(ngavg)
std_perturbLambda_linear_num = p[0]*gaverageseries+p[1]
# axE.plot(gaverageseries,std_perturbLambda_linear,color=cm[2],linestyle='--',linewidth=1.5,alpha=0.5,label='1-order polynomial')
axE.plot(gaverageseries,std_perturbLambda_linear_theo,color='darkgreen',linewidth=1.5,label='analytical')
axE.plot(gaverageseries,-std_perturbLambda_linear_theo,color='darkgreen',linewidth=1.5,label='analytical')
print('std delta_lambda numerical:',p[0])
axE.set_title(r'$\sigma_{\lambda}$')

'''
plt and ax
'''
ylims=[-0.15,0.15]
xlims=[-0.0,1.1]
yticks = np.linspace(-0.15,0.15,2)
xticks = np.linspace(0.0,1.0,2)
axE.set_xlim(xlims)
axE.set_xticks(xticks)
axE.set_ylim(ylims)
axE.set_yticks(yticks)
# ax.legend()

'''
right eigenvector
'''
## calculating the statistics of perturbation lambda
# Coding uses only one point
std_perturb_Reig_numSE,std_perturb_Reig_theo=np.zeros(2),np.zeros(2)
# eigenvectors correspond to g=1
finalReig,finalLeig = np.squeeze(mvecseries[ngavg-1,:,:,0]),np.squeeze(nvecseries[ngavg-1,:,:,0])
stdfinalR,stdfinalL = np.zeros((ntrial,2)),np.zeros((ntrial,2))
stdfinalR[:,0],stdfinalR[:,1]=np.std(finalReig[:,:NE],axis=1),np.std(finalReig[:,NE:],axis=1)
stdfinalL[:,0],stdfinalL[:,1]=np.std(finalLeig[:,:NE],axis=1),np.std(finalLeig[:,NE:],axis=1)
stdtotalR,stdtotalL = np.zeros((ngavg,ntrial,2)),np.zeros((ngavg,ntrial,2))
for ig in range(ngavg):
    stdtotalR[ig,:,0],stdtotalR[ig,:,1] = np.std(np.squeeze(mvecseries[ig,:,:NE,0]),axis=1),np.std(np.squeeze(mvecseries[ig,:,NE:,0]),axis=1)
    stdtotalL[ig,:,0],stdtotalL[ig,:,1] = np.std(np.squeeze(nvecseries[ig,:,:NE,0]),axis=1),np.std(np.squeeze(nvecseries[ig,:,NE:,0]),axis=1)


std_perturb_Reig_numSE[0],std_perturb_Reig_numSE[1] = np.mean(stdfinalR[:,0]),np.mean(stdfinalR[:,1])
std_perturb_Reig_theo[0],std_perturb_Reig_theo[1] = np.sqrt(((xee)**2+(xei)**2)/2/eigvAm[0]**2),np.sqrt(((xie)**2+(xii)**2)/(2)/eigvAm[0]**2)
## theoretical curve for standard deviation of the eigenvalue
std_perturbReig_linear_theo,std_perturbReig_linear_numSE=np.zeros((ngavg,2)),np.zeros((ngavg,2))
std_perturbReig_linear_theo[:,0]=std_perturb_Reig_theo[0]*gaverageseries
std_perturbReig_linear_theo[:,1]=std_perturb_Reig_theo[1]*gaverageseries
std_perturbReig_linear_numSE[:,0]=std_perturb_Reig_numSE[0]*gaverageseries
std_perturbReig_linear_numSE[:,1]=std_perturb_Reig_numSE[1]*gaverageseries

print('std of perturbation right eigenvector, num:',std_perturb_Reig_numSE,'; theo:',std_perturb_Reig_theo)
'''
Variance of the perturbations in eigenvalue lambda
\delta lambda =\Delta lambda = (1) unit change in g
'''
std_perturb_Reig_num,std_perturbReig_linear_num=np.zeros((ngavg,2)),np.zeros((ngavg,2))
std_perturb_Reig_num[:,0],std_perturb_Reig_num[:,1] = np.mean(stdtotalR[:,:,0],axis=1),np.mean(stdtotalR[:,:,1],axis=1)

figR0,axR0=plt.subplots(2,1,figsize=(4,4))
cm='rbg'
# >>>>>>>> numerical solution
axR0[0].scatter(gaverageseries,std_perturb_Reig_num[:,0],color='darkred',label='numerical')
axR0[1].scatter(gaverageseries,std_perturb_Reig_num[:,1],color='darkblue',label='numerical')
for i in range(ntrial):
    axR0[0].scatter(gaverageseries,stdtotalR[:,i,0],s=10,c='darkred',marker='^',label='numerical')
    axR0[1].scatter(gaverageseries,stdtotalR[:,i,1],s=10,c='darkblue',marker='^',label='numerical')
# >>>>>>> Theoretical solutions
axR0[0].plot(gaverageseries,std_perturbReig_linear_theo[:,0],color='red',linewidth=1.5,label='analytical')
axR0[1].plot(gaverageseries,std_perturbReig_linear_theo[:,1],color='blue',linewidth=1.5,label='analytical')
axR0[0].set_title(r'$\sigma_{m}^E$')
axR0[1].set_title(r'$\sigma_{m}^I$')

ylims=[-0.00,1.0]
xlims=[-0.0,1.1]
yticks = np.linspace(0.0,1.0,2)
xticks = np.linspace(0.0,1.0,2)
for i in range(2):
    axR0[i].set_xlim(xlims)
    axR0[i].set_xticks([])
    axR0[i].set_ylim(ylims)
    axR0[i].set_yticks(yticks)
axR0[1].set_xticks(xticks)

'''
left eigenvector
'''
## calculating the statistics of perturbation lambda
# Coding uses only one point
std_perturb_Leig_numSE,std_perturb_Leig_theo=np.zeros(2),np.zeros(2)
std_perturb_Leig_numSE[0],std_perturb_Leig_numSE[1] = np.mean(stdfinalL[:,0]),np.mean(stdfinalL[:,1])
std_perturb_Leig_theo[0],std_perturb_Leig_theo[1] = np.sqrt(2*((JE*xee)**2+(JI*xie)**2)/eigvAm[0]**2),np.sqrt(2*((JE*xei)**2+(JI*xii)**2)/eigvAm[0]**2)
## theoretical curve for standard deviation of the eigenvalue
std_perturbLeig_linear_theo,std_perturbLeig_linear_numSE=np.zeros((ngavg,2)),np.zeros((ngavg,2))
std_perturbLeig_linear_theo[:,0]=std_perturb_Leig_theo[0]*gaverageseries ### !!!
std_perturbLeig_linear_theo[:,1]=std_perturb_Leig_theo[1]*gaverageseries ### !!!
std_perturbLeig_linear_numSE[:,0]=std_perturb_Leig_numSE[0]*gaverageseries
std_perturbLeig_linear_numSE[:,1]=std_perturb_Leig_numSE[1]*gaverageseries

print('std of perturbation left eigenvector, num:',std_perturb_Leig_numSE,'; theo:',std_perturb_Leig_theo)
'''
Variance of the perturbations in eigenvalue lambda
\delta lambda =\Delta lambda = (1) unit change in g
'''
std_perturb_Leig_num,std_perturbLeig_linear_num=np.zeros((ngavg,2)),np.zeros((ngavg,2))
std_perturb_Leig_num[:,0],std_perturb_Leig_num[:,1] = np.mean(stdtotalL[:,:,0],axis=1),np.mean(stdtotalL[:,:,1],axis=1)

figL0,axL0=plt.subplots(2,1,figsize=(4,4))
cm='rbg'
# >>>>>>>> numerical solution
axL0[0].scatter(gaverageseries,std_perturb_Leig_num[:,0],color='darkred',label='numerical')
axL0[1].scatter(gaverageseries,std_perturb_Leig_num[:,1],color='darkblue',label='numerical')
for i in range(ntrial):
    axL0[0].scatter(gaverageseries,stdtotalL[:,i,0],s=10,c='darkred',marker='^',label='numerical')
    axL0[1].scatter(gaverageseries,stdtotalL[:,i,1],s=10,c='darkblue',marker='^',label='numerical')
# >>>>>>> Theoretical solutions
axL0[0].plot(gaverageseries,std_perturbLeig_linear_theo[:,0],color='red',linewidth=1.5,label='analytical')
axL0[1].plot(gaverageseries,std_perturbLeig_linear_theo[:,1],color='blue',linewidth=1.5,label='analytical')
axL0[0].set_title(r'$\sigma_{n}^E$')
axL0[1].set_title(r'$\sigma_{n}^I$')

ylims=[-0.00,3.0]
xlims=[-0.0,1.1]
yticks = np.linspace(0.0,3.0,2)
xticks = np.linspace(0.0,1.0,2)
for i in range(2):
    axL0[i].set_xlim(xlims)
    axL0[i].set_xticks([])
    axL0[i].set_ylim(ylims)
    axL0[i].set_yticks(yticks)
axL0[1].set_xticks(xticks)

## For Case 3 and 4, with symmetric connectivity
* Based on determinant lemma, calculate eigenvalue outlier (spectrum of eigenvalues)
* Based on perturbation theorem, calculate the statistics of eigenvectors (eigenvector perturbations, MEAN)
* Validate the results, in the case with homogeneous random gain and the case with heterogeneous random gain 

In [ ]:
'''
WITH SYMMETRY
'''
'''
Cases 3,4 with symmetry
Focus on mean 
Focus on second moment
Focus on derivative on 
'''
# generate mean matrix
nrank=1
Nt=np.array([750,750])
NE,NI=Nt[0],Nt[1]
N=NE+NI
Nparams=np.array([NE,NI])
'''
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
'''
nrank,ntrial,neta,ngavg=1,20,10,10

## for all casesq
xticks = np.linspace(-4.0,3.0,5)
xlims = [-4.1,3.1]
yticks = np.linspace(-1.3,1.3,3)
ylims = [-1.31,1.31]
theta = np.linspace(0, 2 * np.pi, 200)

''' C0 rules '''
Xtest = np.zeros((NE*2,NE*2))
Xtest[:NE,:NE]=1
idxEE,idyEE=np.where(Xtest>0)
lowinEE=np.where(idxEE<idyEE)
upinEE=np.where(idxEE>idyEE)
idxUpEE,idyUpEE,idxDownEE,idyDownEE=idxEE[lowinEE],idyEE[lowinEE],idxEE[upinEE],idyEE[upinEE]
Xrep = np.zeros_like(Xtest)
Xrep[idxUpEE,idyUpEE]=1
idxUpII,idyUpII,idxDownII,idyDownII=idxUpEE+NE,idyUpEE+NE,idxDownEE+NE,idyDownEE+NE

''' ## Three \bar{J} cases '''
JI,JE,a,b=3.5,1.0,0,0
JEE,JIE,JEI,JII=JE+a,JE-a,JI-b,JI+b
''' Am -- J(g0=0), xAm(g0=0), yAm(g0=0) '''
Am,Jsv=generate_meanmat_eig(Nparams,JEE,JIE,JEI,JII)
# calculate the original \bar{m} and \bar{n}
eigvAm,eigveclAm,eigvecrAm=scpla.eig(Am,left=True,right=True)
meigvecAm=eigvecrAm.copy()
neigvecAm=eigveclAm.copy()
for i in range(nrank):
    normalizecoe=(np.sum(neigvecAm[:,i]*meigvecAm[:,i]))
    neigvecAm[:,i]*=(eigvAm[i]/normalizecoe)
for i in range(1,NE*2):
    normalizecoe=(np.sum(neigvecAm[:,i]*meigvecAm[:,i]))
    neigvecAm[:,i]/=normalizecoe
''' for reference '''
xAm,yAm=np.reshape(meigvecAm[:,0],(NE*2,1)),np.reshape(neigvecAm[:,0].copy()/eigvAm[0],(NE*2,1))

'''
Network Setting for Iterating Following
'''
## SIGNEAT -- SYM/ANTISYM
signeta=np.ones(3)
# signeta[1]*=(-1)
## heterogeneous random gain
''' >>>>>>>>>>>>>> Case 1 and Case 2 >>>>>>>>>>>>>>>>>>>>>>> '''
xee,xei,xie,xii=1.0,1.0,1.0,1.0 #
# coeffeta=np.array([1.0,0.2,0.6]) ## HETERogeneous symmetry D_{\rho}
# coeffeta=np.array([1.0,0.8,1.0])
coeffeta=np.array([1.0,1.0,1.0])
gaverage=1.0

''' Recording Variables '''
# M,N -- with lambda
mvecseries,nvecseries=np.zeros((ngavg,ntrial,NE*2),dtype=complex),np.zeros((ngavg,ntrial,NE*2),dtype=complex)
Reigvecseries,Leigvecseries=np.zeros((ngavg,ntrial,NE*2),dtype=complex),np.zeros((ngavg,ntrial,NE*2),dtype=complex)
Beigvseries=np.zeros((ngavg,ntrial,N),dtype=complex)
''' All random matrices for each trials '''
## Not X total (symmetry), nut Y independent
gYbasexee=np.zeros((ntrial,NE*2,NE*2),dtype=complex)

''' Iterative Processing '''
etaseries=np.linspace(0.0,1.0,neta)
for iktrial in range(ntrial):
    '''
    ##>>>>>>>>>>> g0!=0, >>>>>>>>
    '''
    Xsym=np.random.randn(NE*2,NE*2)*gaverage/np.sqrt(2*NE) # /np.sqrt(2) no later
    Xsym[:NE,:NE]= np.random.randn(NE,NE)*gaverage/np.sqrt(2*NE)
    Xsym[:NE,NE:]= np.random.randn(NE,NE)*gaverage/np.sqrt(2*NE)
    Xsym[NE:,:NE]= np.random.randn(NE,NE)*gaverage/np.sqrt(2*NE)
    Xsym[NE:,NE:]= np.random.randn(NE,NE)*gaverage/np.sqrt(2*NE)
    XsymT=Xsym.copy().T
    X0=Xsym.copy()  # always make mistake here
    J0=Am.copy()+X0.copy()
    ''' # calculate the original \bar{m} and \bar{n} '''
    eigvJ0,eigrvecJ0=la.eig(J0)
    inveigrvecJ0=la.inv(eigrvecJ0)
    meig0 = np.squeeze(eigrvecJ0[:,:].copy())
    neig0 = np.squeeze(inveigrvecJ0[:,:].copy()) # inverse
    neig0=neig0.copy().T
    print('test normal:',np.sum(neig0[:,0]*meig0[:,0]))
    for i in range(nrank):
        neig0[:,i]*=eigvJ0[i]
    for j in range(nrank,N):
        neig0[:,j]*=eigvJ0[j]

    leig0, reig0 = np.zeros((N,N)),np.zeros((N,N))
    ''' The first '''
    reig0=meig0.copy()
    normval=np.sum(reig0*neig0.copy(),axis=0)
    normval=np.repeat(np.reshape(normval,(1,N)),N,axis=0)
    leig0=neig0.copy()/normval.copy()

    print('test normal 2:',np.sum(leig0[:,0]*reig0[:,0]))
    print('test normal 3:',np.sum(meig0[:,0]*neig0[:,0]))
    print('eigenvalues:',eigvJ0[:nrank])
    '''## for reference '''
    x0,y0=np.reshape(reig0[:,0],(NE*2,1)),np.reshape(leig0[:,0].copy(),(NE*2,1))
    ''' ## normalization 0 (reference) absolute reference is xAm, yAm '''
    tildex,tildey=np.reshape(x0[:,0].copy(),(NE*2,1)),np.reshape(y0[:,0].copy(),(NE*2,1))
    ## make sure the E in x is positive, if negative then all x and y should change sign
    if (np.mean(tildey[:NE,0])*yAm[0,0])<0:   # keep the same
        tildex*=(-1)
        tildey*=(-1)
    x0,y0=np.reshape(tildex,(NE*2,1)),np.reshape(tildey,(NE*2,1))
    x0=np.sqrt(np.squeeze(tildey.T@xAm)/np.squeeze(yAm.T@tildex))*tildex
    y0=np.sqrt(np.squeeze(tildex.T@yAm)/np.squeeze(xAm.T@tildey))*tildey
    x0,y0=np.reshape(x0,(NE*2,1)),np.reshape(y0,(NE*2,1))
    for idxeta,eta in enumerate(etaseries):
        etaset=eta*np.ones(3)#np.zeros(3)#eta*np.ones(3)
        for icoeff in range(3):
          etaset[icoeff]*=coeffeta[icoeff]
        # etaset[0]*=0.1
        Xinit = Xsym.copy()

        ''' heterogeneous ETA method 1 '''
        ## EE ##
        asqr=(1-np.sqrt(1-etaset[0]**2))/2.0 ## when eta = 0, asqr = 0, aamp = 0, XT-0, X-1
        aamp=np.sqrt(asqr)
        Xinit[:NE,:NE]=signeta[0]*aamp*XsymT[:NE,:NE].copy()+np.sqrt(1-aamp**2)*Xsym[:NE,:NE].copy()
        ## EI IE##
        asqr=(1-np.sqrt(1-etaset[1]**2))/2.0
        aamp=np.sqrt(asqr)
        Xinit[NE:,:NE]=signeta[1]*aamp*XsymT[NE:,:NE].copy()+np.sqrt(1-aamp**2)*Xsym[NE:,:NE].copy()
        Xinit[:NE,NE:]=signeta[1]*aamp*XsymT[:NE,NE:].copy()+np.sqrt(1-aamp**2)*Xsym[:NE,NE:].copy()
        ## II ##
        asqr=(1-np.sqrt(1-etaset[2]**2))/2.0
        aamp=np.sqrt(asqr)
        Xinit[NE:,NE:]=signeta[2]*aamp*XsymT[NE:,NE:].copy()+np.sqrt(1-aamp**2)*Xsym[NE:,NE:].copy()
        # ### >>>>>>>>. half Exc sym >>>>>>>>>.
        # hNE=int(5*NE/3)
        # asqr=(1-np.sqrt(1-eta**2))/2.0 ## when eta = 0, asqr = 0, aamp = 0, XT-0, X-1
        # aamp=np.sqrt(asqr)
        # Xinit[:hNE,:hNE]=signeta[0]*aamp*XsymT[:hNE,:hNE].copy()+np.sqrt(1-aamp**2)*Xsym[:hNE,:hNE].copy()
        ''' THEORETICAL FULL ETA '''
        symtheofull=gaverage**2*(etaset[0]*signeta[0]+etaset[2]*signeta[2]+2*etaset[1]*signeta[1])/4.0#+(a-b)**2/(NE*2)
        X=Xinit.copy()
        # incase heterogeneous
        X[:NE,:NE]*=(xee)
        X[:NE,NE:]*=(xei)
        X[NE:,:NE]*=(xie)
        X[NE:,NE:]*=(xii)

        # overall
        J = X.copy()+Am.copy()
        JT = J.copy().T
        symnumfull=np.sum(np.sum(J*JT))/(4*NE**2)-np.mean(np.mean(J.copy()))**2
        symnumfull*=2*NE
        print('compare degree of symmetry:',symnumfull,'--num; ',etaset*signeta,'--setting')
        # properties
        eigvJ,eigrvecJ=la.eig(J)
        inveigrvecJ=la.inv(eigrvecJ)
        meig = np.squeeze(eigrvecJ[:,:].copy())
        # meig = np.reshape(meig,(NE*2,nrank))
        neig = np.squeeze(inveigrvecJ[:,:].copy()) # inverse
        neig=neig.copy().T
        print('test normal 4:',np.sum(neig[:,0]*meig[:,0]))
        for i in range(nrank):
            neig[:,i]*=eigvJ[i]
        for j in range(nrank,N):
            neig[:,j]*=eigvJ[j]

        '''  Sort Eigenvalue in ascending    '''
        eigenvalue_Rsort = np.squeeze(-np.abs(np.real(eigvJ.copy())))
        idxsort=np.argsort(eigenvalue_Rsort)
        eigenvalue_sort = np.squeeze(eigvJ.copy())
        # eigvalseries[idxeta,iktrial,:] = eigenvalue_sort[idxsort]
        eigvJ= eigvJ[idxsort]  #### >>>>>>>>>>>>>> resorting >>>>>>>>>
        print('perturbed eigenvalue:',eigvJ[:nrank])
        meig = meig[:,idxsort] #### >>>>>>>>>>>>>> resorting >>>>>>>>>
        neig = neig[:,idxsort] #### >>>>>>>>>>>>>> resorting >>>>>>>>>
        ## >>> for eigendecomposition 
        Leig, Reig = np.zeros((N,N)),np.zeros((N,N))
        Reig = meig.copy()
        Leig = neig.copy()
        normval=np.sum(Reig*Leig.copy(),axis=0)
        normval=np.repeat(np.reshape(normval,(1,N)),N,axis=0)
        Leig=Leig.copy()/normval.copy()  # note that here we change from neig --> Leig, otherwise, if we use copy, there'll be some sort of mistakes
        print('test normal 5:',np.sum(Leig[:,0]*Reig[:,0]))
        '''  normalization 0 (reference)  '''
        tildex,tildey=np.reshape(Reig[:,0].copy(),(NE*2,1)),np.reshape(Leig[:,0].copy(),(NE*2,1))

        if (np.mean(tildey[:NE,0])*yAm[0,0])<0:
            tildex*=(-1)
            tildey*=(-1)
        xnorm0=np.sqrt(np.squeeze(tildey.T@x0)/np.squeeze(y0.T@tildex))*tildex
        ynorm0=np.sqrt(np.squeeze(tildex.T@y0)/np.squeeze(x0.T@tildey))*tildey
        ## l,r without lambda
        Beigvseries[idxeta,iktrial,:]=eigvJ.copy()
        Reigvecseries[idxeta,iktrial,:],Leigvecseries[idxeta,iktrial,:]=xnorm0[:,0].copy(),ynorm0[:,0].copy()
        ## m,n with lambda 
        mvecseries[idxeta,iktrial,:],nvecseries[idxeta,iktrial,:]=xnorm0[:,0].copy()*np.sqrt(2*NE),ynorm0[:,0].copy()*np.sqrt(NE*2)*eigvJ[0]
        ''' record the random derivative '''
        if idxeta == 0:
            print('recording!!!')
            ## log: there is a bug, X is the total random xconn, including symmetry
            gYbasexee[iktrial,:,:]=Xsym.copy()


* record data

In [ ]:
# saving data
now = time.strftime("%Y-%m-%d-%H_%M_%S",time.localtime(time.time()))
Jparameters = np.zeros(4+1)
Jparameters[0],Jparameters[1],Jparameters[2],Jparameters[3]=JEE,JEI,JIE,JII
Jparameters[4]=NE
Randgain=np.zeros(4)
Randgain[0],Randgain[1],Randgain[2],Randgain[3]=xee,xei,xie,xii

np.savez(now+'_perturbationHOMOeta_data',xAm=xAm,yAm=yAm,Jparameters=Jparameters,Randgain=Randgain,gaverage=gaverage,etaseries=etaseries,signeta=signeta,Reigvecseries=Reigvecseries[:,:,:],Leigvecseries=Leigvecseries[:,:,:],Beigvseries=Beigvseries[:,:,:3],meigvecAm=meigvecAm[:,0],neigvecAm=neigvecAm[:,0],eigvAm=eigvAm,mvecseries=mvecseries,nvecseries=nvecseries,gYbasexee=np.real(gYbasexee),coeffeta=coeffeta)

* eigenspectra

In [ ]:
'''
with symmetry, 
eigvJ0
'''
idxgavg,idxtrial=0,8 # 6,3,0
# figtspt,axtspt=plt.subplots(figsize=(6,3))
cm='bgc'
idxc=2
axtspt.scatter(np.real(Beigvseries[idxgavg,idxtrial,1:]),np.imag(Beigvseries[idxgavg,idxtrial,1:]),s=5,c=cm[idxc],alpha=0.25) # >>>>>>>>>>>>>>
axtspt.scatter(np.real(Beigvseries[idxgavg,:,0]),np.imag(Beigvseries[idxgavg,:,0])-0.2*idxc,s=20,c=cm[idxc],marker='^',alpha=0.5) # >>>>>>>>>>>>
axtspt.set_aspect('equal')
axtspt.plot(np.ones(2)*eigvAm[0],np.array([-1.0,1.0]),linestyle='--',linewidth=1.5,color='grey')
axtspt.spines['right'].set_color('none')
axtspt.spines['top'].set_color('none')
# X axis location
axtspt.xaxis.set_ticks_position('bottom')
axtspt.spines['bottom'].set_position(('data', 0))

aee,aei,aie,aii=xee,xei,xie,xii
print(xee,xei,xie,xii)
idxtrial=10
eta=0
xticks = np.linspace(-1.3,1.3,2)
xlims = [-1.3,1.3]
yticks = np.linspace(-1.3,1.3,2)
ylims = [-1.31,1.31]
theta = np.linspace(0, 2 * np.pi, 200)
# first do not multiply at
ahomo=gaverage
xee_,xei_,xie_,xii_=ahomo*aee/np.sqrt(2),ahomo*aei/np.sqrt(2),ahomo*aie/np.sqrt(2),ahomo*aii/np.sqrt(2)
gmat = np.array([[xee_**2,xei_**2],[xie_**2,xii_**2]])
gaverage_=0
for i in range(2):
    for j in range(2):
        gaverage_+=gmat[i,j]/2 # ntype=2
gaverage_=np.sqrt(gaverage_)
eigvgm,eigvecgm=la.eig(gmat) 
r_g2=np.max(eigvgm)
r_g = np.sqrt(r_g2)
print(r_g)
## >>> Now we have symmetry >>
eta=etaseries[idxgavg]
longaxis,shortaxis=(1+eta)*r_g ,(1-eta)*r_g 
xr = longaxis*np.cos(theta)
yr = shortaxis*np.sin(theta)
axtspt.plot(xr, yr, color="gray", linewidth=2.5,linestyle='--',label=r'ellipse') # >>>>>
axtspt.plot(longaxis*np.ones(2), np.array([-1.0,1.0]), color="gray", linewidth=2.5,linestyle='--') # >>>>>
axtspt.set_title(r'$\eta=$'+np.str(eta))
xticks = np.linspace(-3.5,2.5,2)
xlims = [-3.5,2.5]
yticks = np.linspace(-1.0,1.0,2)
ylims = [-1.1,1.1]
axtspt.set_xlim(xlims)
axtspt.set_ylim(ylims)
axtspt.set_xticks(xticks)
axtspt.set_yticks(yticks)
axtspt.set_aspect('equal')

axtspt.plot(xr, yr, color="gray", linewidth=2.5,linestyle='--',label=r'ellipse') # >>>>>
axtspt.plot(longaxis*np.ones(2), np.array([-1.0,1.0]), color="gray", linewidth=2.5,linestyle='--') # >>>>>
axtspt.set_title(r'$\eta=$'+np.str(eta))
xticks = np.linspace(-3.5,2.5,2)
xlims = [-3.5,2.5]
yticks = np.linspace(-1.0,1.0,2)
ylims = [-1.1,1.1]
axtspt.set_xlim(xlims)
axtspt.set_ylim(ylims)
axtspt.set_xticks(xticks)
axtspt.set_yticks(yticks)
axtspt.set_aspect('equal')

* theoretical eigenvalue

In [ ]:
'''
determinant lemma, odd terms 
even order terms should be maitained
'''
xticks = np.linspace(-2.91,-2.5,2)
xlims = [-2.91,-2.5]
yticks = np.linspace(-2.91,-2.5,2)
ylims = [-2.91,-2.5]

coeff=coeffeta.copy()
nAm=np.reshape(yAm*eigvAm[0],(NE*2,1)) # lambda*left eigenvector -- n not l
mAm=np.reshape(xAm.copy(),(NE*2,1))
nAm,mAm=np.real(nAm),np.real(mAm)
lambda_theo2 = np.zeros(neta)
lambda_theo4 = np.zeros(neta)
lambda_num = np.mean(np.squeeze(Beigvseries[:,:,0]),axis=1)
cutoff = 2

for idxeta,eta in enumerate(etaseries):
    etaE,etaB,etaI=eta*coeff[0],eta*coeff[1],eta*coeff[2]
    '''
    Even-order terms have basic element XX.T(X.TX)
    '''
    bXXT=np.eye(NE*2)
    bXXT[:NE,:NE]*=(etaE*signeta[0]+etaB*signeta[1])/4
    bXXT[NE:,NE:]*=(etaI*signeta[2]+etaB*signeta[1])/4
    # cut off = 2
    C=comb(2,1)
    print('second moment item-num.:',C)
    B2=C*(nAm.T@la.matrix_power(bXXT,1)@mAm)*gaverage**2. ## gaverage to the 2
    coeff2 = [1,-eigvAm[0],0,-B2]  ## lambda_0 -- eigvAm
    C=comb(4,2)
    print('fourth moment item-num.:',C)
    B4=C*(nAm.T@la.matrix_power(bXXT,2)@mAm)*gaverage**4 ## gaverage to the 4
    coeff4 = [1,-eigvAm[0],0,-B2,0,-B4]
    ''' soluter np.roots '''
    # lambda_test2=np.roots(coeff2)
    # lambda_theo2[idxeta]=lambda_test2[0]
    ''' Cubic Equation '''
    roots = Cubic(coeff2)
    lambda_theo2[idxeta]=np.real(roots[0])
    lambda_test4=np.roots(coeff4)
    lambda_theo4[idxeta]=lambda_test4[0]
## >>>>> eigenvalue outlier V.S. degree of symmetry
fig,ax=plt.subplots(figsize=(5,3))
ax.plot(etaseries,lambda_theo2[:],'c',label='2 cut off')
ax.plot(etaseries,lambda_theo4[:],'g',label='2 cut off')
ax.plot(etaseries,lambda_num,'b',label='2 cut off')
ax.set_aspect('equal')

In [ ]:
''' THREE SOLUTIONS '''
'''
determinant lemma, odd terms 
even order terms should be maitained
combination

!!! In sparse condition
'''
import numpy.polynomial.polynomial as poly
from scipy.special import comb, perm
coeffB4 = np.zeros(neta)
envelopeRe = np.zeros(neta)

yticks = np.linspace(-2.50,1.5,2)
ylims = [-2.6,1.6]
xticks = np.linspace(-0.,1.,2)
xlims = [-0.1,1.1]

cm='bgr'
nAm=np.reshape(yAm.copy()*eigvAm[0],(NE*2,1))
mAm=np.reshape(xAm.copy(),(NE*2,1))
nAm,mAm=np.real(nAm),np.real(mAm)
lambda_theo2 = np.zeros((neta,3),dtype=complex)
nAmsparse,mAmsparse = nAm.copy(),mAm.copy()
lambda_num = np.transpose(np.squeeze(Beigvseries[:,:,:2]),(2,1,0))
for idxeta,eta in enumerate(etaseries[:]):
    etaset = eta*np.ones(3)
    for i in range(3):
        etaset[i]*=coeffeta[i]
    '''
    Even-order terms have basic element XX.T(X.TX)
    '''
    bXXTE,bXXTI=np.eye(NE),np.eye(NE)
    bXXTE*=(etaset[0]*signeta[0]+etaset[1]*signeta[1])/4
    bXXTI*=(etaset[1]*signeta[1]+ etaset[2]*signeta[2])/4
    # cut off = 2
    C=comb(2,1)
    print(C)
    moment2=np.zeros((NE*2,NE*2))
    moment2[:NE,:NE]=(bXXTE)
    moment2[NE:,NE:]=bXXTI
    B2=np.squeeze(C*(nAm.T@(moment2)@mAm)*gaverage**2)
    roots = Cubic([1,-eigvAm[0],0,-B2])
    coeffB4[idxeta]=-B2
    envelopeRe[idxeta]=np.mean(-np.abs(np.squeeze(Beigvseries[idxeta,:,2])))
    lambda_theo2[idxeta,:]=roots


fig,ax=plt.subplots(figsize=(4,4))  
for i in range(3):
    ax.plot(etaseries,lambda_theo2[:,i],color=cm[i],linewidth=1.5,label='theo')
for i in range(ntrial):
    ax.scatter(etaseries,lambda_num[0,i,:],c='grey',s=5,marker='^',label='2 cut off')
for i in range(ntrial):
    ax.scatter(etaseries,lambda_num[1,i,:],c='black',s=5,marker='o',label='2 cut off')
ax.set_xlabel(r'$\eta$')

fig,ax=plt.subplots(figsize=(4,4))  
for i in range(3):
    ax.plot(coeffB4,lambda_theo2[:,i],color=cm[i],linewidth=1.5,label='theo')
for i in range(ntrial):
    ax.scatter(coeffB4,lambda_num[0,i,:],c='grey',s=5,marker='^',label='2 cut off')
for i in range(ntrial):
    ax.scatter(coeffB4,lambda_num[1,i,:],c='black',s=5,marker='o',label='2 cut off')

ax.set_xlabel(r'$d$')

* overlap --$\Delta_{g_0},\ \Delta_{\rho}$

In [ ]:
'''
Total Theoretical Solution for Covariance
'''

'''
in the case with reciprical connectivity,
with certain degree of homogeneous symmetry

we integrating from 0 to eta and get the CHANGING 
'''
DeltaLR_matheta,DeltaLR_mathg,DeltaLR_num = np.zeros((neta,3),dtype=complex),np.zeros((neta,3),dtype=complex),np.zeros((neta,ntrial,3),dtype=complex)
DeltaLR_eigv = np.zeros((neta,ntrial),dtype=complex)

Jinit = np.zeros((ntrial,NE*2,NE*2),dtype=complex)
## Jinit same for identical trial # realization
Leigvecinit,Reigvecinit = np.zeros((ntrial,NE*2),dtype=complex),np.zeros((ntrial,NE*2),dtype=complex)

for itrial in range(ntrial):
    # initial eigenvectors
    l0,r0=np.reshape(Leigvecseries[0,itrial,:].copy(),(2*NE,1)),np.reshape(Reigvecseries[0,itrial,:].copy(),(2*NE,1))#np.reshape(-yAm,(2*NE,1)),np.reshape(-xAm,(2*NE,1))#

    for idxeta in range(neta):
        ## choose one specific degree of symmetry
        eta = etaseries[idxeta]
        etaE,etaEI,etaI=eta*coeffeta[0],eta*coeffeta[1],eta*coeffeta[2]
        rhoE,rhoEI,rhoI= (1-np.sqrt(1-etaE**2))/2.0,(1-np.sqrt(1-etaEI**2))/2.0,(1-np.sqrt(1-etaI))/2.0
        print('etaset:',etaE,etaEI,etaI)

        print('rhoset:',rhoE,rhoEI,rhoI)

        # numerical value
        changeL_num = np.reshape(np.squeeze(Leigvecseries[idxeta,itrial,:].copy()),(NE*2,1))-np.reshape(yAm,(2*NE,1))#-l0#np.mean(np.squeeze(Leigvecseries[idxeta,itrial,:]))#np.reshape(np.squeeze(Leigvecseries[0,itrial,:]),(NE*2,1))
        # changeL_num[:numE1,0]-=np.mean(changeL_num[:numE1,0])
        # changeL_num[numE1:NE,0]-=np.mean(changeL_num[numE1:NE,0])
        # changeL_num[NE:,0]-=np.mean(changeL_num[NE:,0])
        changeR_num = np.reshape(np.squeeze(Reigvecseries[idxeta,itrial,:].copy()),(NE*2,1))-np.reshape(xAm,(2*NE,1))#-r0#np.mean(np.squeeze(Reigvecseries[idxeta,itrial,:]))#np.reshape(np.squeeze(Reigvecseries[0,itrial,:]),(NE*2,1))
        # changeR_num[:numE1,0]-=np.mean(changeR_num[:numE1,0])
        # changeR_num[numE1:NE,0]-=np.mean(changeR_num[numE1:NE,0])
        # changeR_num[NE:,0]-=np.mean(changeR_num[NE:,0])

        DeltaLR_num[idxeta,itrial,2]=np.squeeze(changeL_num.T@changeR_num)
        DeltaLR_num[idxeta,itrial,0]=np.sum(changeL_num[:NE,0]*changeR_num[:NE,0])
        DeltaLR_num[idxeta,itrial,1]=np.sum(changeL_num[NE:,0]*changeR_num[NE:,0])
   
        # EXCITATORY
        # JE*gaverage**2/(JE-JI)/(eigvAm[0]**2)*(ppercent[0]**2)*(np.sqrt((1-rhoE)*rhoE)-np.sqrt(rhoE)    
        # -JI*gaverage**2/(JE-JI)/(eigvAm[0]**2)*(ppercent[1]**2)*(np.sqrt((1-rhoB)*rhoB)-np.sqrt(rhoB)        bXXTE1,bXXTE2,bXXTII=np.eye(numE1),np.eye(numE2),np.eye(NE)

        '''
        ### >>>>>>>>>>> delta in eta
        '''       
        DeltaLR_matheta[idxeta,0]=JE*gaverage**2/(JE-JI)/(eigvAm[0]**2)*(etaE-2.0*np.sqrt(rhoE))*signeta[0]/4.0
        DeltaLR_matheta[idxeta,0]+=(-JI*gaverage**2/(JE-JI)/(eigvAm[0]**2)*(etaEI-2.0*np.sqrt(rhoEI))*signeta[1]/4.0)
        print('DeltaE1:',DeltaLR_matheta[idxeta,0])

        DeltaLR_matheta[idxeta,1]=JE*gaverage**2/(JE-JI)/(eigvAm[0]**2)*(etaEI-2.0*np.sqrt(rhoEI))*signeta[1]/4.0 
        DeltaLR_matheta[idxeta,1]+=(-JI*gaverage**2/(JE-JI)/(eigvAm[0]**2)*(etaI-2.0*np.sqrt(rhoI))*signeta[2]/4.0)
        '''
        ### >>>>>>>>>>> delta in g with eta
        '''  
        DeltaLR_mathg[idxeta,0]=JE*gaverage**2/(JE-JI)/(eigvAm[0]**2)*(np.sqrt(rhoE))*signeta[0]/2.0
        DeltaLR_mathg[idxeta,0]+=(-JI*gaverage**2/(JE-JI)/(eigvAm[0]**2)*(np.sqrt(rhoEI))*signeta[1]/2.0)
        print('DeltaE1_g:',DeltaLR_mathg[idxeta,0])


        DeltaLR_mathg[idxeta,1]=JE*gaverage**2/(JE-JI)/(eigvAm[0]**2)*(np.sqrt(rhoEI))*signeta[1]/2.0 
        DeltaLR_mathg[idxeta,1]+=(-JI*gaverage**2/(JE-JI)/(eigvAm[0]**2)*(np.sqrt(rhoI))*signeta[2]/2.0)

        DeltaLR_matheta[idxeta,2]=2*DeltaLR_matheta[idxeta,1]+2*DeltaLR_matheta[idxeta,0]
        DeltaLR_mathg[idxeta,2]=2*DeltaLR_mathg[idxeta,1]+2*DeltaLR_mathg[idxeta,0]

        DeltaLR_eigv[idxeta,itrial]= (Beigvseries[idxeta,itrial,0]-eigvAm[0])/eigvAm[0]

In [ ]:
'''
m,n 
'''
DeltaLR_matheta,DeltaLR_mathg,DeltaLR_num = np.zeros((neta,3),dtype=complex),np.zeros((neta,3),dtype=complex),np.zeros((neta,ntrial,3),dtype=complex)
DeltaLR_eigv = np.zeros((neta,ntrial),dtype=complex)

Jinit = np.zeros((ntrial,NE*2,NE*2),dtype=complex)
## Jinit same for identical trial # realization
Leigvecinit,Reigvecinit = np.zeros((ntrial,NE*2),dtype=complex),np.zeros((ntrial,NE*2),dtype=complex)

for itrial in range(ntrial):
    # initial eigenvectors
    l0,r0=np.reshape(nvecseries[0,itrial,:].copy(),(2*NE,1)),np.reshape(mvecseries[0,itrial,:].copy(),(2*NE,1))#np.reshape(-yAm,(2*NE,1)),np.reshape(-xAm,(2*NE,1))#

    for idxeta in range(neta):
        eigvaluem = Beigvseries[idxeta,itrial,0]
        ## choose one specific degree of symmetry
        eta = etaseries[idxeta]
        etaE,etaEI,etaI=eta*coeffeta[0],eta*coeffeta[1],eta*coeffeta[2]
        rhoE,rhoEI,rhoI= (1-np.sqrt(1-etaE**2))/2.0,(1-np.sqrt(1-etaEI**2))/2.0,(1-np.sqrt(1-etaI))/2.0
        print('etaset:',etaE,etaEI,etaI)

        print('rhoset:',rhoE,rhoEI,rhoI)

        # numerical value
        changeL_num = np.reshape(np.squeeze(nvecseries[idxeta,itrial,:].copy()),(NE*2,1))
        changeL_num[:NE,0]-=np.mean(changeL_num[:NE,0])
        changeL_num[NE:,0]-=np.mean(changeL_num[NE:,0])
        changeR_num = np.reshape(np.squeeze(mvecseries[idxeta,itrial,:].copy()),(NE*2,1))
        changeR_num[:NE,0]-=np.mean(changeR_num[:NE,0])
        changeR_num[NE:,0]-=np.mean(changeR_num[NE:,0])

        DeltaLR_num[idxeta,itrial,2]=np.squeeze(changeL_num.T@changeR_num)
        DeltaLR_num[idxeta,itrial,0]=np.sum(changeL_num[:NE,0]*changeR_num[:NE,0])
        DeltaLR_num[idxeta,itrial,1]=np.sum(changeL_num[NE:,0]*changeR_num[NE:,0])
   
        # EXCITATORY
        # JE*gaverage**2/(JE-JI)/(eigvAm[0]**2)*(ppercent[0]**2)*(np.sqrt((1-rhoE)*rhoE)-np.sqrt(rhoE)    
        # -JI*gaverage**2/(JE-JI)/(eigvAm[0]**2)*(ppercent[1]**2)*(np.sqrt((1-rhoB)*rhoB)-np.sqrt(rhoB)        bXXTE1,bXXTE2,bXXTII=np.eye(numE1),np.eye(numE2),np.eye(NE)

        '''
        ### >>>>>>>>>>> delta in eta
        '''       
        DeltaLR_matheta[idxeta,0]=JE*gaverage**2/(JE-JI)/(eigvAm[0]**2)*(etaE-2.0*np.sqrt(rhoE))*signeta[0]/4.0
        DeltaLR_matheta[idxeta,0]+=(-JI*gaverage**2/(JE-JI)/(eigvAm[0]**2)*(etaEI-2.0*np.sqrt(rhoEI))*signeta[1]/4.0)
        DeltaLR_matheta[idxeta,0]*=(NE*2*eigvaluem)
        print('DeltaE1:',DeltaLR_matheta[idxeta,0])

        DeltaLR_matheta[idxeta,1]=JE*gaverage**2/(JE-JI)/(eigvAm[0]**2)*(etaEI-2.0*np.sqrt(rhoEI))*signeta[1]/4.0 
        DeltaLR_matheta[idxeta,1]+=(-JI*gaverage**2/(JE-JI)/(eigvAm[0]**2)*(etaI-2.0*np.sqrt(rhoI))*signeta[2]/4.0)
        DeltaLR_matheta[idxeta,1]*=(NE*2*eigvaluem)
        
        '''
        ### >>>>>>>>>>> delta in g with eta
        '''  
        DeltaLR_mathg[idxeta,0]=JE*gaverage**2/(JE-JI)/(eigvAm[0]**2)*(np.sqrt(rhoE))*signeta[0]/2.0
        DeltaLR_mathg[idxeta,0]+=(-JI*gaverage**2/(JE-JI)/(eigvAm[0]**2)*(np.sqrt(rhoEI))*signeta[1]/2.0)
        print('DeltaE1_g:',DeltaLR_mathg[idxeta,0])
        DeltaLR_mathg[idxeta,0]*=(NE*2*eigvaluem)

        DeltaLR_mathg[idxeta,1]=JE*gaverage**2/(JE-JI)/(eigvAm[0]**2)*(np.sqrt(rhoEI))*signeta[1]/2.0 
        DeltaLR_mathg[idxeta,1]+=(-JI*gaverage**2/(JE-JI)/(eigvAm[0]**2)*(np.sqrt(rhoI))*signeta[2]/2.0)
        DeltaLR_mathg[idxeta,1]*=(NE*2*eigvaluem)

        DeltaLR_matheta[idxeta,2]=2*DeltaLR_matheta[idxeta,1]+2*DeltaLR_matheta[idxeta,0]
        DeltaLR_mathg[idxeta,2]=2*DeltaLR_mathg[idxeta,1]+2*DeltaLR_mathg[idxeta,0]

        DeltaLR_eigv[idxeta,itrial]= (Beigvseries[idxeta,itrial,0]-eigvAm[0])*(NE*2)

In [ ]:
fig,ax=plt.subplots(figsize=(5,5))
cl='rbk'
for i in range(ntrial):
    ax.scatter(etaseries,np.squeeze(DeltaLR_num[:,i,2])/N,s=15,c='blue',marker='^',alpha=0.5)
    ax.scatter(etaseries+0.01,np.squeeze(DeltaLR_eigv[:,i])/N,s=15,c='red',marker='o',alpha=0.5)
# ax.plot(etaseries,np.mean(np.squeeze(DeltaLR_num[:,:,2]),axis=1)/N,color=cl[0],linewidth=1.5)
# ax.plot(etaseries,np.mean(np.squeeze(DeltaLR_eigv[:,:]),axis=1)/N,color=cl[1],linewidth=1.5)
ax.plot(etaseries,DeltaLR_matheta[:,2]/N+DeltaLR_mathg[:,2]/N,color=cl[2],linewidth=1.5)

* loading pattern

In [ ]:
'''
loading patterns
for ensemble-averaging results
>>>>>>>>>>>>>>>>> with eigenvalues >>>>>>>>>>>>>>>
'''
xticks = np.linspace(-0.4,0.45,2)
xlims = [-0.4,0.45]
yticks = np.linspace(-0.4,0.45,2)
ylims = [-0.4,0.45]
# xticks = np.linspace(-0.2,0.25,2)
# xlims = [-0.2,0.25]
# yticks = np.linspace(-0.2,0.25,2)
# ylims = [-0.2,0.25]
xticks = np.linspace(-0.15,0.20,2)
xlims = [-0.15,0.20]
yticks = np.linspace(-0.05,0.10,2)
ylims = [-0.05,0.1]
theta = np.linspace(0, 2 * np.pi, 200)
''' how eigenvalue outlier changes with g_bar '''

idxtrial=9#16#
idxeta=9#9# 
alphaval=0.10
edgv='black'
cm='br'
fig,ax=plt.subplots(figsize=(5,3))  
nrank=1
ntrial = np.shape(Reigvecseries)[1]
NE=int(NE)
print(NE)
'''loading vector changing'''
meanmE,meanmI,meannE,meannI=np.zeros(nrank),np.zeros(nrank),np.zeros(nrank),np.zeros(nrank)

mEvec,mIvec,nEvec,nIvec=np.squeeze(Reigvecseries[idxeta,:,:NE]),np.squeeze(Reigvecseries[idxeta,:,NE:]),np.squeeze(Leigvecseries[idxeta,:,:NE]),np.squeeze(Leigvecseries[idxeta,:,NE:])
mEvec,mIvec,nEvec,nIvec=mEvec.flatten(),mIvec.flatten(),nEvec.flatten(),nIvec.flatten()
scale_std=3.0
for irank in range(nrank):
    meanmEtotal,stdmEtotal = np.mean(mEvec),np.std(mEvec)
    varmE = mEvec - meanmEtotal
    idxwhere = np.where(np.abs(varmE)>scale_std*stdmEtotal)
    mEvec[idxwhere]=meanmEtotal
    meanmE[irank]=np.mean(mEvec)
    # puring I
    meanmItotal,stdmItotal = np.mean(mIvec),np.std(mIvec)
    varmI = mIvec - meanmItotal
    idxwhere = np.where(np.abs(varmI)>scale_std*stdmItotal)
    mIvec[idxwhere]=meanmItotal
    meanmI[irank]=np.mean(mIvec)
    
    # n vector
    # puring E
    meannEtotal,stdnEtotal = np.mean(nEvec),np.std(nEvec)
    varnE = nEvec - meannEtotal
    idxwhere = np.where(np.abs(varnE)>scale_std*stdnEtotal)
    nEvec[idxwhere]=meannEtotal
    meannE[irank]=np.mean(nEvec)

    # puring I
    meannItotal,stdnItotal = np.mean(nIvec),np.std(nIvec)
    varnI = nIvec - meannItotal
    idxwhere = np.where(np.abs(varnI)>scale_std*stdnItotal)
    nIvec[idxwhere]=meannItotal
    meannI[irank]=np.mean(nIvec)

''' calculate directions '''
noiseE,noiseI = np.zeros((NE*ntrial,2,nrank)),np.zeros((NE*ntrial,2,nrank))
dirvecE,dirvecI=np.zeros((2,2)),np.zeros((2,2))
for irank in range(nrank):
    # E 0 M 1 N
    noiseE[:,0,irank],noiseE[:,1,irank]= nEvec-meannE[irank],mEvec-meanmE[irank]
    noiseI[:,0,irank],noiseI[:,1,irank]= nIvec-meannI[irank],mIvec-meanmI[irank]
# m1n1
covdirE,covdirI=np.squeeze(noiseE[:,:,0]).T@np.squeeze(noiseE[:,:,0]),np.squeeze(noiseI[:,:,0]).T@np.squeeze(noiseI[:,:,0])
_,dirvecE=la.eig(covdirE)
_,dirvecI=la.eig(covdirI)  

eigvuse = np.real(Beigvseries[idxeta,idxtrial,0])
ax.scatter(nIvec*np.sqrt(N)*eigvuse,mIvec*np.sqrt(N),s=5.0,c='green',alpha=alphaval)#cm[1],alpha=alphaval)
ax.scatter(nEvec*np.sqrt(N)*eigvuse,mEvec*np.sqrt(N),s=5.0,c='r',alpha=alphaval)#cm[0],alpha=alphaval)

ax.plot([meannE[0],meannE[0]+dirvecE[0,0]],[meanmE[0],meanmE[0]+dirvecE[1,0]],color=edgv,linestyle='--',linewidth=1.5)
ax.plot([meannE[0],meannE[0]+dirvecE[0,1]],[meanmE[0],meanmE[0]+dirvecE[1,1]],color=edgv,linestyle='--',linewidth=1.5)
ax.plot([meannI[0],meannI[0]+dirvecI[0,0]],[meanmI[0],meanmI[0]+dirvecI[1,0]],color=edgv,linestyle='--',linewidth=1.5)
ax.plot([meannI[0],meannI[0]+dirvecI[0,1]],[meanmI[0],meanmI[0]+dirvecI[1,1]],color=edgv,linestyle='--',linewidth=1.5)

xticks = np.linspace(-15,10,2)
xlims = [-15,10]
yticks = np.linspace(-1,4,2)
ylims = [-1,4]

confidence_ellipse(np.real(nEvec),np.real(mEvec),ax,edgecolor=edgv)
confidence_ellipse(np.real(nIvec),np.real(mIvec),ax,edgecolor=edgv)
ax.axvline(c='grey', lw=1)
ax.axhline(c='grey', lw=1)
ax.set_xlim(xlims)
ax.set_ylim(ylims)
ax.set_xticks(xticks)
ax.set_yticks(yticks)
ax.set_aspect('equal')


In [11]:
meigvecAm=eigvecrAm.copy()
neigvecAm=eigveclAm.copy()
for i in range(nrank):
    normalizecoe=(np.sum(neigvecAm[:,i]*meigvecAm[:,i]))
    neigvecAm[:,i]*=(eigvAm[i]/normalizecoe)
for i in range(1,NE*2):
    normalizecoe=(np.sum(neigvecAm[:,i]*meigvecAm[:,i]))
    neigvecAm[:,i]/=normalizecoe

'''
m n
>>>>>>>>>>>>>>>>> with eigenvalues >>>>>>>>>>>>>>>
'''
xticks = np.linspace(-15,10,2)
xlims = [-15,10]
yticks = np.linspace(-1,4,2)
ylims = [-1,4]

theta = np.linspace(0, 2 * np.pi, 200)
''' how eigenvalue outlier changes with g_bar '''

idxtrial=9#16#
idxeta=9#9# 
alphaval=0.10
edgv='black'
cm='br'
fig,ax=plt.subplots(figsize=(5,3))  
nrank=1
ntrial = np.shape(Reigvecseries)[1]
NE=int(NE)
print(NE)
'''loading vector changing'''
meanmE,meanmI,meannE,meannI=np.zeros(nrank),np.zeros(nrank),np.zeros(nrank),np.zeros(nrank)

mEvec,mIvec,nEvec,nIvec=np.squeeze(meigvecAm[:NE,0])*np.sqrt(NE*2),np.squeeze(meigvecAm[NE:,0])*np.sqrt(NE*2),np.squeeze(neigvecAm[:NE,0])*np.sqrt(NE*2),np.squeeze(neigvecAm[NE:,0])*np.sqrt(NE*2)
scale_std=3.0
for irank in range(nrank):
    meanmE[irank]=np.mean(mEvec)
    # puring I
    meanmI[irank]=np.mean(mIvec)
    
    # n vector
    # puring E
    meannE[irank]=np.mean(nEvec)

    # puring I
    meannI[irank]=np.mean(nIvec)

ax.scatter(nIvec,mIvec,s=25.0,c='blue',alpha=alphaval)#cm[1],alpha=alphaval)
ax.scatter(nEvec,mEvec,s=25.0,c='red',alpha=alphaval)#cm[0],alpha=alphaval)

ax.axvline(c='grey', lw=1)
ax.axhline(c='grey', lw=1)
ax.set_xlim(xlims)
ax.set_ylim(ylims)
ax.set_xticks(xticks)
ax.set_yticks(yticks)
ax.set_aspect('equal')


750


<ipython-input-11-d0ed4274775a>:38: ComplexWarning: Casting complex values to real discards the imaginary part
  meanmE[irank]=np.mean(mEvec)
<ipython-input-11-d0ed4274775a>:40: ComplexWarning: Casting complex values to real discards the imaginary part
  meanmI[irank]=np.mean(mIvec)
<ipython-input-11-d0ed4274775a>:44: ComplexWarning: Casting complex values to real discards the imaginary part
  meannE[irank]=np.mean(nEvec)
<ipython-input-11-d0ed4274775a>:47: ComplexWarning: Casting complex values to real discards the imaginary part
  meannI[irank]=np.mean(nIvec)


In [9]:
'''
m n
>>>>>>>>>>>>>>>>> with eigenvalues >>>>>>>>>>>>>>>
'''
xticks = np.linspace(-15,10,2)
xlims = [-15,10]
yticks = np.linspace(-1,4,2)
ylims = [-1,4]

theta = np.linspace(0, 2 * np.pi, 200)
''' how eigenvalue outlier changes with g_bar '''

idxtrial=9#16#
idxeta=9#9# 
alphaval=0.10
edgv='black'
cm='br'
fig,ax=plt.subplots(figsize=(5,3))  
nrank=1
ntrial = np.shape(Reigvecseries)[1]
NE=int(NE)
print(NE)
'''loading vector changing'''
meanmE,meanmI,meannE,meannI=np.zeros(nrank),np.zeros(nrank),np.zeros(nrank),np.zeros(nrank)

mEvec,mIvec,nEvec,nIvec=np.squeeze(mvecseries[idxeta,:,:NE]),np.squeeze(mvecseries[idxeta,:,NE:]),np.squeeze(nvecseries[idxeta,:,:NE]),np.squeeze(nvecseries[idxeta,:,NE:])
mEvec,mIvec,nEvec,nIvec=mEvec.flatten(),mIvec.flatten(),nEvec.flatten(),nIvec.flatten()
scale_std=3.0
for irank in range(nrank):
    meanmEtotal,stdmEtotal = np.mean(mEvec),np.std(mEvec)
    varmE = mEvec - meanmEtotal
    idxwhere = np.where(np.abs(varmE)>scale_std*stdmEtotal)
    mEvec[idxwhere]=meanmEtotal
    meanmE[irank]=np.mean(mEvec)
    # puring I
    meanmItotal,stdmItotal = np.mean(mIvec),np.std(mIvec)
    varmI = mIvec - meanmItotal
    idxwhere = np.where(np.abs(varmI)>scale_std*stdmItotal)
    mIvec[idxwhere]=meanmItotal
    meanmI[irank]=np.mean(mIvec)
    
    # n vector
    # puring E
    meannEtotal,stdnEtotal = np.mean(nEvec),np.std(nEvec)
    varnE = nEvec - meannEtotal
    idxwhere = np.where(np.abs(varnE)>scale_std*stdnEtotal)
    nEvec[idxwhere]=meannEtotal
    meannE[irank]=np.mean(nEvec)

    # puring I
    meannItotal,stdnItotal = np.mean(nIvec),np.std(nIvec)
    varnI = nIvec - meannItotal
    idxwhere = np.where(np.abs(varnI)>scale_std*stdnItotal)
    nIvec[idxwhere]=meannItotal
    meannI[irank]=np.mean(nIvec)

''' calculate directions '''
noiseE,noiseI = np.zeros((NE*ntrial,2,nrank)),np.zeros((NE*ntrial,2,nrank))
dirvecE,dirvecI=np.zeros((2,2)),np.zeros((2,2))
for irank in range(nrank):
    # E 0 M 1 N
    noiseE[:,0,irank],noiseE[:,1,irank]= nEvec-meannE[irank],mEvec-meanmE[irank]
    noiseI[:,0,irank],noiseI[:,1,irank]= nIvec-meannI[irank],mIvec-meanmI[irank]
# m1n1
covdirE,covdirI=np.squeeze(noiseE[:,:,0]).T@np.squeeze(noiseE[:,:,0]),np.squeeze(noiseI[:,:,0]).T@np.squeeze(noiseI[:,:,0])
_,dirvecE=la.eig(covdirE)
_,dirvecI=la.eig(covdirI) 
for i in range(2):
    for j in range(2):
        dirvecE[i,j]*=(1*np.sqrt(N))
        dirvecI[i,j]*=(1*np.sqrt(N))

ax.scatter(nIvec,mIvec,s=5.0,c='blue',alpha=alphaval)#cm[1],alpha=alphaval)
ax.scatter(nEvec,mEvec,s=5.0,c='red',alpha=alphaval)#cm[0],alpha=alphaval)

ax.plot([meannE[0],meannE[0]+dirvecE[0,0]],[meanmE[0],meanmE[0]+dirvecE[1,0]],color=edgv,linestyle='--',linewidth=1.5)
ax.plot([meannE[0],meannE[0]+dirvecE[0,1]],[meanmE[0],meanmE[0]+dirvecE[1,1]],color=edgv,linestyle='--',linewidth=1.5)
ax.plot([meannI[0],meannI[0]+dirvecI[0,0]],[meanmI[0],meanmI[0]+dirvecI[1,0]],color=edgv,linestyle='--',linewidth=1.5)
ax.plot([meannI[0],meannI[0]+dirvecI[0,1]],[meanmI[0],meanmI[0]+dirvecI[1,1]],color=edgv,linestyle='--',linewidth=1.5)

confidence_ellipse(np.real(nEvec),np.real(mEvec),ax,edgecolor=edgv)
confidence_ellipse(np.real(nIvec),np.real(mIvec),ax,edgecolor=edgv)
ax.axvline(c='grey', lw=1)
ax.axhline(c='grey', lw=1)
ax.set_xlim(xlims)
ax.set_ylim(ylims)
ax.set_xticks(xticks)
ax.set_yticks(yticks)
ax.set_aspect('equal')


750
(2, 2)
(2, 2)


<ipython-input-9-dff02a5262f5>:34: ComplexWarning: Casting complex values to real discards the imaginary part
  meanmE[irank]=np.mean(mEvec)
<ipython-input-9-dff02a5262f5>:40: ComplexWarning: Casting complex values to real discards the imaginary part
  meanmI[irank]=np.mean(mIvec)
<ipython-input-9-dff02a5262f5>:48: ComplexWarning: Casting complex values to real discards the imaginary part
  meannE[irank]=np.mean(nEvec)
<ipython-input-9-dff02a5262f5>:55: ComplexWarning: Casting complex values to real discards the imaginary part
  meannI[irank]=np.mean(nIvec)
<ipython-input-9-dff02a5262f5>:62: ComplexWarning: Casting complex values to real discards the imaginary part
  noiseE[:,0,irank],noiseE[:,1,irank]= nEvec-meannE[irank],mEvec-meanmE[irank]
<ipython-input-9-dff02a5262f5>:63: ComplexWarning: Casting complex values to real discards the imaginary part
  noiseI[:,0,irank],noiseI[:,1,irank]= nIvec-meannI[irank],mIvec-meanmI[irank]
